In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import linearmodels as lm
import matplotlib
import math
import statsmodels.formula.api as smf
import statsmodels.api as sm

from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import FirstDifferenceOLS
from linearmodels.panel import compare
from matplotlib import pyplot as plt

from pandas.api.types import is_numeric_dtype

pd.set_option('display.max_columns', 500)

%matplotlib inline

/Users/rianajawad/anaconda3/lib/python3.7/site-packages/linearmodels/panel/data.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Categorical, DataFrame, Index, MultiIndex, Panel, Series,


In [192]:
df=pd.read_csv('final_estimation_file_apta_added.csv',
                    encoding='utf-8')
df.drop(columns=['Unnamed: 0'],inplace=True)

In [193]:
tsd_data=pd.read_csv('tsd_pop_employment.csv',
                    encoding='utf-8')
tsd_data.drop(columns=['Unnamed: 0','CBSA Title'],inplace=True)
tsd_data.head()

,CBSA Code,STATE,Year,TOT_EMP_TSD_TRACTS,TOT_EMP_ALL_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS
0,10180,Texas,2002,9489.0,57166.0,58397.0,0.0,0.0,0.0,23086.0,25800.0,9511.0,48886.0,58397.0,0.0,0.0,1216.0,2355.0,0.0,0.0,0.0,954.0,1065.0,336.0,2019.0,2355.0,0.0,0.0
1,10180,Texas,2003,9484.0,56917.0,57221.0,0.0,0.0,0.0,22049.0,25074.0,10098.0,47123.0,57221.0,0.0,0.0,1216.0,2462.0,0.0,0.0,0.0,1032.0,1069.0,361.0,2101.0,2462.0,0.0,0.0
2,10180,Texas,2004,9599.0,59025.0,61419.0,0.0,0.0,0.0,23062.0,27199.0,11158.0,50261.0,61419.0,0.0,0.0,1216.0,2970.0,0.0,0.0,0.0,1231.0,1315.0,424.0,2546.0,2970.0,0.0,0.0
3,10180,Texas,2005,9645.0,57879.0,60001.0,0.0,0.0,0.0,21854.0,26081.0,12066.0,47935.0,60001.0,0.0,0.0,1216.0,2938.0,0.0,0.0,0.0,1134.0,1326.0,478.0,2460.0,2938.0,0.0,0.0
4,10180,Texas,2006,9819.0,59588.0,62311.0,0.0,0.0,0.0,22008.0,26639.0,13664.0,48647.0,62311.0,0.0,0.0,1216.0,2983.0,0.0,0.0,0.0,1122.0,1342.0,519.0,2464.0,2983.0,0.0,0.0


In [194]:
df=pd.merge(df,tsd_data,left_on=['CBSA','Year'],right_on=['CBSA Code','Year'],how='left')
df=df.drop_duplicates()
df.head()

,CBSA,MNAME,Mode,Year,UPT_ADJUSTED,VRM_ADJUSTED,FARE_ADJUSTED,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,AVG_FARE_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOT_EMP_TSD_TRACTS,TOT_EMP_ALL_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS
0,10420,"Akron, OH Metro Area",Bus,2002,5226385.660,4026372.388,5.965827e+05,2002,2018,0.114148,NaN,NaN,NaN,646583.7344,496141.0469,137193.6719,-389.53125,25405.20313,26028.32813,26651.07813,7117.03125,682354.11,7.55,8.94,83.85,24.64,62.9,12.46,265452.20,22830.14,58254.52,117083.25,46891.53,20392.77,8.60,43046.82813,55300.5625,39.49,46.56,13.95,21948.42,346147.17,368095.58,5.96,94.04,1.36,NaN,119786.0,14.88,1.40,2014,http://www.crainscleveland.com/article/2014082...,0,0,4004892.28,35567.28438,36439.65938,37311.50938,60265.55938,77420.7875,1.9040,0,0,0,NaN,NaN,0.0,NaN,0,0,0,1,1,H,24,NaN,309488.2031,87.4,6.2,0.8,1.7,0.1,0.7,3.1,Goes to negative when extrapolating. Possible ...,0.159808,2417.239652,both,24,10420.0,Ohio,81463.0,314749.0,305401.0,0.0,0.0,0.0,92556.0,127165.0,85680.0,219721.0,305401.0,0.0,0.0,13619.2,54517.0,0.0,0.0,0.0,18228.0,25397.0,10892.0,43625.0,54517.0,0.0,0.0
1,10420,"Akron, OH Metro Area",Bus,2003,5395901.888,3976568.624,2.318587e+06,2002,2018,0.429694,NaN,NaN,NaN,659397.0625,504631.8125,137582.3125,753.12500,25120.18750,25632.68750,25995.68750,10350.875,682850.56,8.81,8.66,82.79,24.33,63.0,12.68,269144.19,22073.44,67400.94,115443.00,44874.88,19351.94,8.20,43566.68750,56379.7500,39.24,46.66,14.10,23342.50,348574.17,371916.67,6.28,93.72,1.55,NaN,119199.0,14.71,1.37,2014,http://www.crainscleveland.com/article/2014082...,0,0,15690640.83,34414.65688,35116.78188,35614.09188,59686.36188,77240.2575,2.1235,0,0,0,NaN,NaN,0.0,NaN,0,0,0,1,1,H,24,NaN,316497.1875,87.2,6.4,0.8,1.7,0.1,0.7,3.1,Goes to negative when extrapolating. Possible ...,0.588681,2417.305061,both,24,10420.0,Ohio,79781.0,313075.0,318486.0,0.0,0.0,0.0,95379.0,128948.0,94159.0,224327.0,318486.0,0.0,0.0,13619.2,51976.0,0.0,0.0,0.0,17093.0,23767.0,11116.0,40860.0,51976.0,0.0,0.0
2,10420,"Akron, OH Metro Area",Bus,2004,5907162.281,4315392.555,2.279211e+06,2002,2018,0.385839,NaN,NaN,NaN,672492.7500,513183.7500,138058.7500,1942.50000,24804.25000,252

In [195]:
df['TSD_EMP_PCT_LEHD']=100*df['TOT_EMP_TSD_TRACTS']/df['TOT_EMP_ALL_TRACTS']
df['TSD_EMP_PCT_CBSA']=100*df['TOT_EMP_TSD_TRACTS']/df['TOT_EMP_MSA']
df['TSD_POP_PCT_LEHD']=100*df['TOTAL_JOBS_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS'] #RAC data
df['TSD_POP_PCT_CBSA']=100*df['TOTAL_JOBS_RA_TSD_TRACTS']/df['Tot_Pop']
df['TSD_POP_WAGE_UNDER_1250']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']
df['TSD_POP_WAGE_BETN_1250_3333']=100*df['TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']
df['TSD_POP_WAGE_UNDER_3333']=100*df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']

df['TSD_POP_WAGE_UNDER_1250_PCT_RA']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_UNDER_1250_RA_TRACTS']
df['TSD_POP_WAGE_BETN_1250_3333_PCT_RA']=100*df['TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_BETN_1250_3333_RA_TRACTS']
df['TSD_POP_WAGE_UNDER_3333_PCT_RA']=100*df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_UNDER_3333_RA_TRACTS']

df['TSD_POP_WAGE_UNDER_1250_PCT_TSD']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TSD_TRACTS']
df['TSD_POP_WAGE_BETN_1250_3333_PCT_TSD']=100*df['TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TSD_TRACTS']
df['TSD_POP_WAGE_UNDER_3333_PCT_TSD']=100*df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TSD_TRACTS']

In [196]:
# read in and join the maintenance and restructure data
mr=pd.read_csv('maintenance_restructure.csv',
                    encoding='utf-8')
mr = mr[['CBSA', 'Mode', 'Year', 'MAINTENANCE', 'MAINTENANCE_NYC', 'MAINTENANCE_WMATA', 'RESTRUCTURE']]

df = df.merge(mr, on=['CBSA', 'Mode', 'Year'], how='left')

In [197]:
# read in mechanical and other failures

failures=pd.read_csv('maintenance_final.csv',encoding='utf-8')

failures.drop(columns=['Unnamed: 0'],inplace=True)

df=df.merge(failures, on=['CBSA','MNAME','Mode','Year'],how='left')

In [198]:
df=df.rename(columns={'UPT_ADJUSTED':'UPT_ADJ','VRM_ADJUSTED':'VRM_ADJ','FARE_ADJUSTED':'FARE_ADJ','AVG_FARE_2018':'FARE_per_UPT_2018',
                     'Mecha_Failures':'Mechanical_Failures'})

In [199]:
df['Mechanical_Failures']=df['Mechanical_Failures'].fillna(0)
df['Total_Failures']=df['Total_Failures'].fillna(0)
df['MDBF_Mechanical']=np.where(df['Mechanical_Failures']>0,df['VRM_ADJ']/df['Mechanical_Failures'],df['VRM_ADJ'])
df['MDBF_Total']=np.where(df['Total_Failures']>0,df['VRM_ADJ']/df['Total_Failures'],df['VRM_ADJ'])

In [200]:
# read in and join the bike share updates
bsu=pd.read_csv('bike_share_update.csv',
                    encoding='utf-8')
bsu = bsu[['CBSA', 'Mode', 'Year', 'PBS_Flag_Update']]

df = df.merge(bsu, on=['CBSA', 'Mode', 'Year'], how='left')

#df['PBS_Flag_Missing'] = df['PBS_Flag'].apply(lambda x : x.isnan())
df['PBS_Flag'] = np.where(df['PBS_Flag'].isnull(), df['PBS_Flag_Update'], df['PBS_Flag'])

In [201]:
# 4 apta clusers
df['CLUSTER_APTA4'] = np.floor(df['CLUSTER_APTA']/10)

In [202]:
# exclude if ridership is zero--missing data
df = df[df['UPT_ADJ']>0]

In [203]:
# fill in missing data as needed

# min fare is zero
df['FARE_per_UPT_2018'] = df['FARE_per_UPT_2018'].apply(lambda x : max(x,0))

# max fare is $20
df['FARE_per_UPT_2018'] = df['FARE_per_UPT_2018'].apply(lambda x : min(x,20))

# zero vehicle HH can't go negative
df['HH_0Veh'] = df['HH_0Veh'].apply(lambda x : max(x,0))

In [204]:
df=df.drop_duplicates(['CBSA','Mode','Year'])

In [205]:
# exclude problematic data
old_len = len(df)
df = df[df['VRM_ADJ']>0]
new_len = len(df)
print('Excluding ' + str(old_len-new_len) + ' records with missing VRM') 

# exclude if problem flag 2 is set
old_len = len(df)
df = df[df['PROBLEM_FLAG2']!=1]
new_len = len(df)
print('Excluding ' + str(old_len-new_len) + ' records with problem flag set to 1') 




Excluding 2 records with missing VRM
Excluding 36 records with problem flag set to 1


In [206]:
# exclude discontinuous data
df = df.sort_values(by=['CBSA', 'Mode', 'Year'])

excluded_records = 100
while excluded_records > 0: 
    old_len = len(df)
    df['YearDiff'] = df['Year'].shift(-1) - df['Year']
    df = df[(df['YearDiff']==1) | (df['Year']==2018)]
    excluded_records = old_len - len(df)
    print('Excluding ' + str(excluded_records) + ' records with discontinuities in year.') 




Excluding 1 records with discontinuities in year.
Excluding 1 records with discontinuities in year.
Excluding 0 records with discontinuities in year.


In [207]:
df['BUS_FLAG'] = np.where(df['Mode']=='Bus', 1, 0)
df['RAIL_FLAG'] = np.where(df['Mode']=='Rail', 1, 0)

In [208]:
# include population + employment
df['POP_EMP'] = df['Tot_Pop'] + df['TOT_EMP_MSA']
df['HH_EMP'] = df['Total_HH'] + df['TOT_EMP_MSA']


# percent of pop born outside USA
df['Tot_NonUSA_POP_pct'] = df['Tot_NonUSA_POP'] / df['Tot_Pop'] * 100

In [209]:
# time effects
df['YEARS_SINCE_2002']  = df['Year'] - 2002

df['YEARS_2002_2010']  = df['Year'].apply(lambda x : min(x-2002, 8))
df['YEARS_AFTER_2010'] = df['Year'].apply(lambda x : max(x-2010, 0))

df['YEARS_2002_2014']  = df['Year'].apply(lambda x : min(x-2002, 12))
df['YEARS_AFTER_2014'] = df['Year'].apply(lambda x : max(x-2014, 0))


df['YEARS_SINCE_2002_BUS']  = df['YEARS_SINCE_2002'] * df['BUS_FLAG']
df['YEARS_SINCE_2002_RAIL'] = df['YEARS_SINCE_2002'] * df['RAIL_FLAG']

df['YEARS_2002_2010_BUS']   = df['YEARS_2002_2010'] * df['BUS_FLAG']
df['YEARS_AFTER_2010_BUS']  = df['YEARS_AFTER_2010']* df['BUS_FLAG']
df['YEARS_2002_2010_RAIL']  = df['YEARS_2002_2010'] * df['RAIL_FLAG']
df['YEARS_AFTER_2010_RAIL'] = df['YEARS_AFTER_2010']* df['RAIL_FLAG']

df['YEARS_2002_2014_BUS']   = df['YEARS_2002_2014'] * df['BUS_FLAG']
df['YEARS_AFTER_2014_BUS']  = df['YEARS_AFTER_2014']* df['BUS_FLAG']
df['YEARS_2002_2014_RAIL']  = df['YEARS_2002_2014'] * df['RAIL_FLAG']
df['YEARS_AFTER_2014_RAIL'] = df['YEARS_AFTER_2014']* df['RAIL_FLAG']

In [210]:
# more time effects

df['YEARS_AFTER_2012'] = df['Year'].apply(lambda x : max(x-2012, 0))

df['TNC_YEARS_AFTER_2012'] = df['YEARS_AFTER_2012'] * df['TNC_FLAG']

df['TNC_YEARS_AFTER_2012_BUS'] = df['TNC_YEARS_AFTER_2012'] * df['BUS_FLAG']
df['TNC_YEARS_AFTER_2012_RAIL'] = df['TNC_YEARS_AFTER_2012'] * df['RAIL_FLAG']

In [211]:
# test rail ramp up period
df['YEARS_SINCE_RAIL_START'] = (df['Year'] - df['min_year']) * df['RAIL_FLAG']
df['YEARS_SINCE_RAIL_START'] = np.where(df['min_year']==2002, 0, df['YEARS_SINCE_RAIL_START'])

df['YEARS_SINCE_RAIL_START_1'] = df['YEARS_SINCE_RAIL_START'].apply(lambda x : min(x, 1))
df['YEARS_SINCE_RAIL_START_2'] = df['YEARS_SINCE_RAIL_START'].apply(lambda x : min(x, 2))
df['YEARS_SINCE_RAIL_START_3'] = df['YEARS_SINCE_RAIL_START'].apply(lambda x : min(x, 3))



In [212]:
# calculate bus vs rail stuff

# various bus/rail specifics

df['TNC_FLAG_BUS'] = df['TNC_FLAG'] * df['BUS_FLAG']
df['TNC_FLAG_RAIL'] = df['TNC_FLAG'] * df['RAIL_FLAG']

df['YEARS_SINCE_TNC_BUS'] = df['YEARS_SINCE_TNC'] * df['BUS_FLAG']
df['YEARS_SINCE_TNC_RAIL'] = df['YEARS_SINCE_TNC'] * df['RAIL_FLAG']

df['dockless_flag'] = df['dockCt'].apply(lambda x : min(x,1))
df['scooter_flag']  = df['scooterCt'].apply(lambda x : min(x,1))

df['BIKE_SHARE'] = df['PBS_Flag'] + df['dockless_flag']
df['BIKE_SHARE'] = df['BIKE_SHARE'].apply(lambda x : min(x,1))


df['PBS_Flag_BUS']   = df['PBS_Flag']   * df['BUS_FLAG']
df['dockCt_BUS']     = df['dockCt']     * df['BUS_FLAG']
df['docklessCt_BUS'] = df['docklessCt'] * df['BUS_FLAG']
df['scooterCt_BUS']  = df['scooterCt']  * df['BUS_FLAG']
df['dockless_flag_BUS'] = df['dockless_flag'] * df['BUS_FLAG']
df['scooter_flag_BUS']  = df['scooter_flag']  * df['BUS_FLAG']
df['BIKE_SHARE_BUS']  = df['BIKE_SHARE']  * df['BUS_FLAG']


df['PBS_Flag_RAIL']   = df['PBS_Flag']   * df['RAIL_FLAG']
df['dockCt_RAIL']     = df['dockCt']     * df['RAIL_FLAG']
df['docklessCt_RAIL'] = df['docklessCt'] * df['RAIL_FLAG']
df['scooterCt_RAIL']  = df['scooterCt']  * df['RAIL_FLAG']
df['dockless_flag_RAIL'] = df['dockless_flag'] * df['RAIL_FLAG']
df['scooter_flag_RAIL']  = df['scooter_flag']  * df['RAIL_FLAG']
df['BIKE_SHARE_RAIL']  = df['BIKE_SHARE']  * df['RAIL_FLAG']

In [213]:
# convert JTW to numeric columns
df['JTW_DA_PCT']        = df['JTW_DA_PCT'].replace('#VALUE!', np.nan)
df['JTW_CARPOOLED_PCT'] = df['JTW_CARPOOLED_PCT'].replace('#VALUE!', np.nan)
df['JTW_TRANSIT_PCT']   = df['JTW_TRANSIT_PCT'].replace('#VALUE!', np.nan)
df['JTW_WALK_PCT']      = df['JTW_WALK_PCT'].replace('#VALUE!', np.nan)
df['JTW_BICYCLE_PCT']   = df['JTW_BICYCLE_PCT'].replace('#VALUE!', np.nan)
df['JTW_OTHER_PCT']     = df['JTW_OTHER_PCT'].replace('#VALUE!', np.nan)
df['JTW_HOME_PCT']      = df['JTW_HOME_PCT'].replace('#VALUE!', np.nan)


df['JTW_DA_PCT']        = df['JTW_DA_PCT'].replace('N', np.nan)
df['JTW_CARPOOLED_PCT'] = df['JTW_CARPOOLED_PCT'].replace('N', np.nan)
df['JTW_TRANSIT_PCT']   = df['JTW_TRANSIT_PCT'].replace('N', np.nan)
df['JTW_WALK_PCT']      = df['JTW_WALK_PCT'].replace('N', np.nan)
df['JTW_BICYCLE_PCT']   = df['JTW_BICYCLE_PCT'].replace('N', np.nan)
df['JTW_OTHER_PCT']     = df['JTW_OTHER_PCT'].replace('N', np.nan)
df['JTW_HOME_PCT']      = df['JTW_HOME_PCT'].replace('N', np.nan)



In [214]:
# convert JTW to numeric columns
df['JTW_DA_PCT']        = df['JTW_DA_PCT'].astype(float)
df['JTW_CARPOOLED_PCT'] = df['JTW_CARPOOLED_PCT'].astype(float)
df['JTW_TRANSIT_PCT']   = df['JTW_TRANSIT_PCT'].astype(float)
df['JTW_WALK_PCT']      = df['JTW_WALK_PCT'].astype(float)
df['JTW_BICYCLE_PCT']   = df['JTW_BICYCLE_PCT'].astype(float)
df['JTW_OTHER_PCT']     = df['JTW_OTHER_PCT'].astype(float)
df['JTW_HOME_PCT']      = df['JTW_HOME_PCT'].astype(float)

In [215]:
# immigrant population percentage
df['Tot_NonUSA_POP_pct'] = df['Tot_NonUSA_POP'] / df['Tot_Pop'] * 100

In [216]:
# bike share plus scooters for rail
df['BIKE_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL']

# different spec of bike and scooter
df['BIKE_SCOOTER_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] + df['scooter_flag_RAIL']

# different spec of bike and scooter
df['BIKE_SCOOTER_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] + df['scooter_flag_RAIL']

# merge bike and scooter for bus
df['BIKE_SCOOTER_SCOOTER_BUS'] = df['BIKE_SHARE_BUS'] + df['scooter_flag_BUS'] + df['scooter_flag_BUS']

# merge bike and scooter for bus
df['BIKE_SCOOTER_BUS'] = df['BIKE_SHARE_BUS'] + df['scooter_flag_BUS'] 
df['BIKE_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] 

# scooter or bike
df['BIKE_OR_SCOOTER_BUS']  = df['BIKE_SHARE_BUS'] + df['scooter_flag_BUS'] 
df['BIKE_OR_SCOOTER_BUS']  = df['BIKE_OR_SCOOTER_BUS'].apply(lambda x : min(x, 1))

df['BIKE_OR_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] 
df['BIKE_OR_SCOOTER_RAIL'] = df['BIKE_OR_SCOOTER_RAIL'].apply(lambda x : min(x, 1))

# calculate average vehicles
df['AVG_VEHS'] = (df['HH_1Veh'] + 2*df['HH_2Veh'] + 3*df['HH_3Veh'] + 4*df['HH_4+Veh']) / (df['HH_0Veh'] + df['HH_1Veh'] + df['HH_2Veh'] + df['HH_3Veh'] + df['HH_4+Veh'])
df['AVG_VEHS_log'] = np.log(1+df['AVG_VEHS'])

In [217]:
# walk bike other commuting
df['JTW_WALK_BIKE_OTHER_PCT'] = df['JTW_WALK_PCT'] + df['JTW_BICYCLE_PCT'] + df['JTW_OTHER_PCT']

# walk bike commuting
df['JTW_WALK_BIKE_PCT'] = df['JTW_WALK_PCT'] + df['JTW_BICYCLE_PCT']

In [218]:
df['VRM_ADJ_BUS'] = df['VRM_ADJ'] * df['BUS_FLAG']
df['VRM_ADJ_RAIL'] = df['VRM_ADJ'] * df['RAIL_FLAG']


#maintenance effect on competing mode
df['MDBF_Mechanical_RAIL']  = df['MDBF_Mechanical'] * df['RAIL_FLAG']
df['MDBF_Mechanical_BUS'] = df['MDBF_Mechanical']  * df['BUS_FLAG']

df['MDBF_Total_RAIL']  = df['MDBF_Total'] * df['RAIL_FLAG']
df['MDBF_Total_BUS'] = df['MDBF_Total']  * df['BUS_FLAG']

In [219]:
# segment TNCs by cluster
df['HI_OPEX']  = np.where(df['CLUSTER_APTA4']==1, 1, 0)
df['MID_OPEX'] = np.where(df['CLUSTER_APTA4']==2, 1, 0)
df['LOW_OPEX'] = np.where(df['CLUSTER_APTA4']==3, 1, 0)
df['NEW_YORK'] = np.where(df['CLUSTER_APTA4']==10, 1, 0)

df['YEARS_SINCE_TNC_BUS_HI']  = df['YEARS_SINCE_TNC_BUS'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_BUS_MID'] = df['YEARS_SINCE_TNC_BUS'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_BUS_LOW'] = df['YEARS_SINCE_TNC_BUS'] * df['LOW_OPEX']
df['YEARS_SINCE_TNC_BUS_NY']  = df['YEARS_SINCE_TNC_BUS'] * df['NEW_YORK']

df['YEARS_SINCE_TNC_RAIL_HI']  = df['YEARS_SINCE_TNC_RAIL'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_RAIL_MID'] = df['YEARS_SINCE_TNC_RAIL'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_RAIL_LOW'] = df['YEARS_SINCE_TNC_RAIL'] * df['LOW_OPEX']
df['YEARS_SINCE_TNC_RAIL_NY']  = df['YEARS_SINCE_TNC_RAIL'] * df['NEW_YORK']

df['YEARS_SINCE_TNC_BUS_HINY']  = df['YEARS_SINCE_TNC_BUS_HI'] + df['YEARS_SINCE_TNC_BUS_NY']
df['YEARS_SINCE_TNC_RAIL_HINY']  = df['YEARS_SINCE_TNC_RAIL_HI'] + df['YEARS_SINCE_TNC_RAIL_NY']

df['YEARS_SINCE_TNC_BUS_MIDLOW']  = df['YEARS_SINCE_TNC_BUS_MID'] + df['YEARS_SINCE_TNC_BUS_LOW']
df['YEARS_SINCE_TNC_RAIL_MIDLOW']  = df['YEARS_SINCE_TNC_RAIL_MID'] + df['YEARS_SINCE_TNC_RAIL_LOW']

In [220]:
# bikes and scooters by segment

df['BIKE_SHARE_HI']  = df['BIKE_SHARE'] * df['HI_OPEX']
df['BIKE_SHARE_MID'] = df['BIKE_SHARE'] * df['MID_OPEX']
df['BIKE_SHARE_LOW'] = df['BIKE_SHARE'] * df['LOW_OPEX']
df['BIKE_SHARE_NY']  = df['BIKE_SHARE'] * df['NEW_YORK']

df['scooter_flag_HI']  = df['scooter_flag'] * df['HI_OPEX']
df['scooter_flag_MID'] = df['scooter_flag'] * df['MID_OPEX']
df['scooter_flag_LOW'] = df['scooter_flag'] * df['LOW_OPEX']
df['scooter_flag_NY']  = df['scooter_flag'] * df['NEW_YORK']


df['BIKE_SHARE_HINY']  = df['BIKE_SHARE_HI'] + df['BIKE_SHARE_NY']
df['BIKE_SHARE_MIDLOW']  = df['BIKE_SHARE_MID'] + df['BIKE_SHARE_LOW']

df['scooter_flag_HINY']  = df['scooter_flag_HI'] + df['scooter_flag_NY']
df['scooter_flag_MIDLOW']  = df['scooter_flag_MID'] + df['scooter_flag_LOW']

In [221]:
tnc_rev=pd.read_csv('tnc_revenue.csv',encoding='utf-8')
tnc_rev.head()

,Unnamed: 0,CBSA,MNAME,Mode,Year,TNC_FLAG,YEARS_SINCE_TNC,UBER_NET_REVENUE,UBER_REV_SCALED
0,0,10420,"Akron, OH Metro Area",Bus,2014,1,0,445,1.000000
1,1,10420,"Akron, OH Metro Area",Bus,2015,1,1,1690,3.797753
2,2,10420,"Akron, OH Metro Area",Bus,2016,1,2,4964,11.155056
3,3,10420,"Akron, OH Metro Area",Bus,2017,1,3,7932,17.824719
4,4,10420,"Akron, OH Metro Area",Bus,2018,1,4,11270,25.325843


In [222]:
tnc_rev.drop(columns=['Unnamed: 0'],inplace=True)

In [223]:
list(tnc_rev)

['CBSA',
 'MNAME',
 'Mode',
 'Year',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'UBER_NET_REVENUE',
 'UBER_REV_SCALED']

In [224]:
df=pd.merge(df,tnc_rev,on=['CBSA',
 'MNAME',
 'Mode',
 'Year',
 'TNC_FLAG',
 'YEARS_SINCE_TNC'],how='left')
df.head()

,CBSA,MNAME,Mode,Year,UPT_ADJ,VRM_ADJ,FARE_ADJ,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,FARE_per_UPT_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOT_EMP_TSD_TRACTS,TOT_EMP_ALL_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS,TSD_EMP_PCT_LEHD,TSD_EMP_PCT_CBSA,TSD_POP_PCT_LEHD,TSD_POP_PCT_CBSA,TSD_POP_WAGE_UNDER_1250,TSD_POP_WAGE_BETN_1250_3333,TSD_POP_WAGE_UNDER_3333,TSD_POP_WAGE_UNDER_1250_PCT_RA,TSD_POP_WAGE_BETN_1250_3333_PCT_RA,TSD_POP_WAGE_UNDER_3333_PCT_RA,TSD_POP_WAGE_UNDER_1250_PCT_TSD,TSD_POP_WAGE_BETN_1250_3333_PCT_TSD,TSD_POP_WAGE_UNDER_3333_PCT_TSD,MAINTENANCE,MAINTENANCE_NYC,MAINTENANCE_WMATA,RESTRUCTURE,Mechanical_Failures,Other_Failures,Total_Failures,MDBF_Mechanical,MDBF_Total,PBS_Flag_Update,CLUSTER_APTA4,YearDiff,BUS_FLAG,RAIL_FLAG,POP_EMP,HH_EMP,Tot_NonUSA_POP_pct,YEARS_SINCE_2002,YEARS_2002_2010,YEARS_AFTER_2010,YEARS_2002_2014,YEARS_AFTER_2014,YEARS_SINCE_2002_BUS,YEARS_SINCE_2002_RAIL,YEARS_2002_2010_BUS,YEARS_AFTER_2010_BUS,YEARS_2002_2010_RAIL,YEARS_AFTER_2010_RAIL,YEARS_2002_2014_BUS,YEARS_AFTER_2014_BUS,YEARS_2002_2014_RAIL,YEARS_AFTER_2014_RAIL,YEARS_AFTER_2012,TNC_YEARS_AFTER_2012,TNC_YEARS_AFTER_2012_BUS,TNC_YEARS_AFTER_2012_RAIL,YEARS_SINCE_RAIL_START,YEARS_SINCE_RAIL_START_1,YEARS_SINCE_RAIL_START_2,YEARS_SINCE_RAIL_START_3,TNC_FLAG_BUS,TNC_FLAG_RAIL,YEARS_SINCE_TNC_BUS,YEARS_SINCE_TNC_RAIL,dockless_flag,scooter_flag,BIKE_SHARE,PBS_Flag_BUS,dockCt_BUS,docklessCt_BUS,scooterCt_BUS,dockless_flag_BUS,scooter_flag_BUS,BIKE_SHARE_BUS,PBS_Flag_RAIL,dockCt_RAIL,docklessCt_RAIL,scooterCt_RAIL,dockless_flag_RAIL,scooter_flag_RAIL,BIKE_SHARE_RAIL,BIKE_SCOOTER_RAIL,BIKE_SCOOTER_SCOOTER_RAIL,BIKE_SCOOTER_SCOOTER_BUS,BIKE_SCOOTER_BUS,BIKE_OR_SCOOTER_BUS,BIKE_OR_SCOOTER_RAIL,AVG_VEHS,AVG_VEHS_log,JTW_WALK_BIKE_OTHER_PCT,JTW_WALK_BIKE_PCT,VRM_ADJ_BUS,VRM_ADJ_RAIL,MDBF_Mechanical_RAIL,MDBF_Mechanical_BUS,MDBF_Total_RAIL,MDBF_Total_BUS,HI_OPEX,MID_OPEX,LOW_OPEX,NEW_YORK,YEARS_SINCE_TNC_BUS_HI,YEARS_SINCE_TNC_BUS_MID,YEARS_SINCE_TNC_BUS_LOW,YEARS_SINCE_TNC_BUS_NY,YEARS_SINCE_TNC_RAIL_HI,YEARS_SINCE_TNC_RAIL_MID,YEARS_SINCE_TNC_RAIL_LOW,YEARS_SINCE_TNC_RAIL_NY,YEARS_SINCE_TNC_BUS_HINY,YEARS_SINCE_TNC_RAIL_HINY,YEARS_SINCE_TNC_BUS_MIDLOW,Y

In [226]:
df['UBER_NET_REVENUE']=df['UBER_NET_REVENUE'].fillna(0)
df['UBER_REV_SCALED']=df['UBER_REV_SCALED'].fillna(0)
df.head(20)

,CBSA,MNAME,Mode,Year,UPT_ADJ,VRM_ADJ,FARE_ADJ,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,FARE_per_UPT_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOT_EMP_TSD_TRACTS,TOT_EMP_ALL_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS,TSD_EMP_PCT_LEHD,TSD_EMP_PCT_CBSA,TSD_POP_PCT_LEHD,TSD_POP_PCT_CBSA,TSD_POP_WAGE_UNDER_1250,TSD_POP_WAGE_BETN_1250_3333,TSD_POP_WAGE_UNDER_3333,TSD_POP_WAGE_UNDER_1250_PCT_RA,TSD_POP_WAGE_BETN_1250_3333_PCT_RA,TSD_POP_WAGE_UNDER_3333_PCT_RA,TSD_POP_WAGE_UNDER_1250_PCT_TSD,TSD_POP_WAGE_BETN_1250_3333_PCT_TSD,TSD_POP_WAGE_UNDER_3333_PCT_TSD,MAINTENANCE,MAINTENANCE_NYC,MAINTENANCE_WMATA,RESTRUCTURE,Mechanical_Failures,Other_Failures,Total_Failures,MDBF_Mechanical,MDBF_Total,PBS_Flag_Update,CLUSTER_APTA4,YearDiff,BUS_FLAG,RAIL_FLAG,POP_EMP,HH_EMP,Tot_NonUSA_POP_pct,YEARS_SINCE_2002,YEARS_2002_2010,YEARS_AFTER_2010,YEARS_2002_2014,YEARS_AFTER_2014,YEARS_SINCE_2002_BUS,YEARS_SINCE_2002_RAIL,YEARS_2002_2010_BUS,YEARS_AFTER_2010_BUS,YEARS_2002_2010_RAIL,YEARS_AFTER_2010_RAIL,YEARS_2002_2014_BUS,YEARS_AFTER_2014_BUS,YEARS_2002_2014_RAIL,YEARS_AFTER_2014_RAIL,YEARS_AFTER_2012,TNC_YEARS_AFTER_2012,TNC_YEARS_AFTER_2012_BUS,TNC_YEARS_AFTER_2012_RAIL,YEARS_SINCE_RAIL_START,YEARS_SINCE_RAIL_START_1,YEARS_SINCE_RAIL_START_2,YEARS_SINCE_RAIL_START_3,TNC_FLAG_BUS,TNC_FLAG_RAIL,YEARS_SINCE_TNC_BUS,YEARS_SINCE_TNC_RAIL,dockless_flag,scooter_flag,BIKE_SHARE,PBS_Flag_BUS,dockCt_BUS,docklessCt_BUS,scooterCt_BUS,dockless_flag_BUS,scooter_flag_BUS,BIKE_SHARE_BUS,PBS_Flag_RAIL,dockCt_RAIL,docklessCt_RAIL,scooterCt_RAIL,dockless_flag_RAIL,scooter_flag_RAIL,BIKE_SHARE_RAIL,BIKE_SCOOTER_RAIL,BIKE_SCOOTER_SCOOTER_RAIL,BIKE_SCOOTER_SCOOTER_BUS,BIKE_SCOOTER_BUS,BIKE_OR_SCOOTER_BUS,BIKE_OR_SCOOTER_RAIL,AVG_VEHS,AVG_VEHS_log,JTW_WALK_BIKE_OTHER_PCT,JTW_WALK_BIKE_PCT,VRM_ADJ_BUS,VRM_ADJ_RAIL,MDBF_Mechanical_RAIL,MDBF_Mechanical_BUS,MDBF_Total_RAIL,MDBF_Total_BUS,HI_OPEX,MID_OPEX,LOW_OPEX,NEW_YORK,YEARS_SINCE_TNC_BUS_HI,YEARS_SINCE_TNC_BUS_MID,YEARS_SINCE_TNC_BUS_LOW,YEARS_SINCE_TNC_BUS_NY,YEARS_SINCE_TNC_RAIL_HI,YEARS_SINCE_TNC_RAIL_MID,YEARS_SINCE_TNC_RAIL_LOW,YEARS_SINCE_TNC_RAIL_NY,YEARS_SINCE_TNC_BUS_HINY,YEARS_SINCE_TNC_RAIL_HINY,YEARS_SINCE_TNC_BUS_MIDLOW,Y

In [230]:
df['UBER_REV_SCALED_BUS'] = df['UBER_REV_SCALED'] * df['BUS_FLAG']
df['UBER_REV_SCALED_RAIL'] = df['UBER_REV_SCALED'] * df['RAIL_FLAG']

df['UBER_REV_SCALED_BUS_HI']  = df['UBER_REV_SCALED_BUS'] * df['HI_OPEX']
df['UBER_REV_SCALED_BUS_MID'] = df['UBER_REV_SCALED_BUS'] * df['MID_OPEX']
df['UBER_REV_SCALED_BUS_LOW'] = df['UBER_REV_SCALED_BUS'] * df['LOW_OPEX']
df['UBER_REV_SCALED_BUS_NY']  = df['UBER_REV_SCALED_BUS'] * df['NEW_YORK']

df['UBER_REV_SCALED_RAIL_HI']  = df['UBER_REV_SCALED_RAIL'] * df['HI_OPEX']
df['UBER_REV_SCALED_RAIL_MID'] = df['UBER_REV_SCALED_RAIL'] * df['MID_OPEX']
df['UBER_REV_SCALED_RAIL_LOW'] = df['UBER_REV_SCALED_RAIL'] * df['LOW_OPEX']
df['UBER_REV_SCALED_RAIL_NY']  = df['UBER_REV_SCALED_RAIL'] * df['NEW_YORK']

df['UBER_REV_SCALED_BUS_HINY']  = df['UBER_REV_SCALED_BUS_HI'] + df['UBER_REV_SCALED_BUS_NY']
df['UBER_REV_SCALED_RAIL_HINY']  = df['UBER_REV_SCALED_RAIL_HI'] + df['UBER_REV_SCALED_RAIL_NY']

df['UBER_REV_SCALED_BUS_MIDLOW']  = df['UBER_REV_SCALED_BUS_MID'] + df['UBER_REV_SCALED_BUS_LOW']
df['UBER_REV_SCALED_RAIL_MIDLOW']  = df['UBER_REV_SCALED_RAIL_MID'] + df['UBER_REV_SCALED_RAIL_LOW']

In [232]:
df.head()

CBSA      UPT_ADJ      VRM_ADJ      FARE_ADJ  \
ID                       Year                                                  
Akron, OH Metro Area-Bus 2002  10420  5226385.660  4026372.388  5.965827e+05   
                         2003  10420  5395901.888  3976568.624  2.318587e+06   
                         2004  10420  5907162.281  4315392.555  2.279211e+06   
                         2005  10420  6013680.881  4053011.065  2.728754e+06   
                         2006  10420  6070797.419  3994786.013  2.855273e+06   

                               min_year  max_year  FARE_per_UPT  PROBLEM_FLAG  \
ID                       Year                                                   
Akron, OH Metro Area-Bus 2002      2002      2018      0.114148           NaN   
                         2003      2002      2018      0.429694           NaN   
                         2004      2002      2018      0.385839           NaN   
                         2005      2002      2018      0.453758           NaN   
                         2006      2002      2018      0.470329           NaN   

                               PROBLEM_FLAG2      Tot_Pop  Tot_Instate_Pop  \
ID                       Year                                                
Akron, OH Metro Area-Bus 2002            NaN  646583.7344      496141.0469   
                         2003            NaN  659397.0625      504631.8125   
                         2004            NaN  672492.7500      513183.7500   
                         2005            NaN  684459.0000      521491.0000   
                         2006            NaN  700943.0000      530777.0000   

                               Tot_Outstate_Pop  Tot_NonUSA_POP  \
ID                       Year                                     
Akron, OH Metro Area-Bus 2002       137193.6719      -389.53125   
                         2003       137582.3125       753.12500   
                         2004       138058.7500      1942.50000   
                         2005       138184.0000      2945.00000   
                         2006       139714.0000      4695.00000   

                               Total_Median_Income_Individual  \
ID                       Year                                   
Akron, OH Metro Area-Bus 2002                     25405.20313   
                         2003                     25120.18750   
                         2004                     24804.25000   
                         2005                     24612.00000   
                         2006                     23925.00000   

                               Native_Instate_Med_Inc_Indiv  \
ID                       Year                                 
Akron, OH Metro Area-Bus 2002                   26028.32813   
                         2003                   25632.68750   
                         2004                   25208.25000   
                         2005                   24899.00000   
                         2006                   24129.00000   

                               Native_Outstate_Med_Inc_Indiv  \
ID                       Year                                  
Akron, OH Metro Area-Bus 2002                    26651.07813   
                         2003                    25995.68750   
                         2004                    25305.25000   
                         2005                    24755.00000   
                         2006                    23644.00000   

                               Total_Pop_Poverty  Pop_Below100_Poverty  \
ID                       Year                                            
Akron, OH Metro Area-Bus 2002          682354.11                  7.55   
                         2003          682850.56                  8.81   
                         2004          683397.75                 10.08   
                         2005          683742.00                 11.30   
                         2006          684898.00                 12.70   

                    

In [231]:
# set the indices
df['ID'] = df['MNAME'] + '-' + df['Mode']
df=df.set_index(['ID','Year'])

KeyError: 'MNAME'

In [233]:
# keep only the numeric columns -- the estimation will give an error otherwise
df = df.select_dtypes(include=[np.number])

In [234]:
# create a log of all fields
for col in df.columns:
    df[col+'_log'] = np.log(df[col]+1)

In [235]:
# Test TNC ramp up

df['YEARS_SINCE_TNC_BUS_2'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 2))
df['YEARS_SINCE_TNC_BUS_3'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 3))
df['YEARS_SINCE_TNC_BUS_4'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 4))
df['YEARS_SINCE_TNC_BUS_5'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 5))

df['YEARS_SINCE_TNC_RAIL_2'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 2))
df['YEARS_SINCE_TNC_RAIL_3'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 3))
df['YEARS_SINCE_TNC_RAIL_4'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 4))
df['YEARS_SINCE_TNC_RAIL_5'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 5))

In [236]:
# count the first year of TNC presence
df['YEARS_SINCE_TNC_BUS2'] = df['YEARS_SINCE_TNC_BUS'] + df['TNC_FLAG_BUS']
df['YEARS_SINCE_TNC_RAIL2'] = df['YEARS_SINCE_TNC_RAIL'] + df['TNC_FLAG_RAIL']


In [237]:
## start in first year TNCs show up

df['YEARS_SINCE_TNC_BUS2_NY']  = df['YEARS_SINCE_TNC_BUS2'] * df['NEW_YORK']
df['YEARS_SINCE_TNC_BUS2_HI']  = df['YEARS_SINCE_TNC_BUS2'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_BUS2_MID'] = df['YEARS_SINCE_TNC_BUS2'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_BUS2_LOW'] = df['YEARS_SINCE_TNC_BUS2'] * df['LOW_OPEX']

df['YEARS_SINCE_TNC_BUS2_HINY']   = df['YEARS_SINCE_TNC_BUS2_NY'] + df['YEARS_SINCE_TNC_BUS2_HI']
df['YEARS_SINCE_TNC_BUS2_MIDLOW'] = df['YEARS_SINCE_TNC_BUS2_MID'] + df['YEARS_SINCE_TNC_BUS2_LOW']

df['YEARS_SINCE_TNC_RAIL2_NY']  = df['YEARS_SINCE_TNC_RAIL2'] * df['NEW_YORK']
df['YEARS_SINCE_TNC_RAIL2_HI']  = df['YEARS_SINCE_TNC_RAIL2'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_RAIL2_MID'] = df['YEARS_SINCE_TNC_RAIL2'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_RAIL2_LOW'] = df['YEARS_SINCE_TNC_RAIL2'] * df['LOW_OPEX']

df['YEARS_SINCE_TNC_RAIL2_HINY']   = df['YEARS_SINCE_TNC_RAIL2_NY'] + df['YEARS_SINCE_TNC_RAIL2_HI']
df['YEARS_SINCE_TNC_RAIL2_MIDLOW'] = df['YEARS_SINCE_TNC_RAIL2_MID'] + df['YEARS_SINCE_TNC_RAIL2_LOW']

In [238]:
## BIKE SHARE SEGMENTATION

df['BIKE_SHARE_NY']  = df['BIKE_SHARE'] * df['NEW_YORK']
df['BIKE_SHARE_HI']  = df['BIKE_SHARE'] * df['HI_OPEX']
df['BIKE_SHARE_MID'] = df['BIKE_SHARE'] * df['MID_OPEX']
df['BIKE_SHARE_LOW'] = df['BIKE_SHARE'] * df['LOW_OPEX']

df['BIKE_SHARE_HINY']   = df['BIKE_SHARE_NY'] + df['BIKE_SHARE_HI']
df['BIKE_SHARE_MIDLOW'] = df['BIKE_SHARE_MID'] + df['BIKE_SHARE_LOW']

df['scooter_flag_NY']  = df['scooter_flag'] * df['NEW_YORK']
df['scooter_flag_HI']  = df['scooter_flag'] * df['HI_OPEX']
df['scooter_flag_MID'] = df['scooter_flag'] * df['MID_OPEX']
df['scooter_flag_LOW'] = df['scooter_flag'] * df['LOW_OPEX']

df['scooter_flag_HINY']   = df['scooter_flag_NY'] + df['scooter_flag_HI']
df['scooter_flag_MIDLOW'] = df['scooter_flag_MID'] + df['scooter_flag_LOW']



## Model Estimation

In [80]:
# base model 1 from 9-5
### preferred model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6305
Estimator:                   PanelOLS   R-squared (Between):              0.9641
No. Observations:                4145   R-squared (Within):               0.6305
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9647
Time:                        02:47:07   Log-likelihood                    1321.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      549.04
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(12,3861)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             549.04
                            

In [87]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_2018',
 'HH_MEAN_INC_2018',
 'GAS_PRICE_2018',

In [88]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6321
Estimator:                   PanelOLS   R-squared (Between):              0.7418
No. Observations:                4033   R-squared (Within):               0.6321
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.7440
Time:                        11:11:34   Log-likelihood                    1294.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      538.03
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(12,3757)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             538.03
                            

In [89]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6323
Estimator:                   PanelOLS   R-squared (Between):              0.7381
No. Observations:                4033   R-squared (Within):               0.6323
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.7404
Time:                        11:11:41   Log-likelihood                    1295.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      496.90
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(13,3756)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             496.90
                            

In [90]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6323
Estimator:                   PanelOLS   R-squared (Between):              0.7383
No. Observations:                4033   R-squared (Within):               0.6323
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.7406
Time:                        11:11:43   Log-likelihood                    1295.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      538.38
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(12,3757)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             538.38
                            

In [91]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)
#total population and employment
#emoloyment at TSDs, and density of low-wage population on ridership
#after including POP_EMP_log, coeff on TSD_POP_WAGE_UNDER_1250 jumps up
#Should carry this model forward

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6354
Estimator:                   PanelOLS   R-squared (Between):              0.9486
No. Observations:                4033   R-squared (Within):               0.6354
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9489
Time:                        11:11:49   Log-likelihood                    1312.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      503.47
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(13,3756)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             503.47
                            

In [92]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_WAGE_UNDER_1250_PCT_RA \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6346
Estimator:                   PanelOLS   R-squared (Between):              0.9390
No. Observations:                4033   R-squared (Within):               0.6346
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9394
Time:                        11:13:00   Log-likelihood                    1308.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      501.83
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(13,3756)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             501.83
                            

In [242]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_2018',
 'HH_MEAN_INC_2018',
 'GAS_PRICE_2018',

In [243]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_WAGE_UNDER_1250_PCT_RA \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + UBER_REV_SCALED_BUS_HINY \
                    + UBER_REV_SCALED_BUS_MIDLOW \
                    + UBER_REV_SCALED_RAIL_HINY \
                    + UBER_REV_SCALED_RAIL_MIDLOW \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6350
Estimator:                   PanelOLS   R-squared (Between):              0.9253
No. Observations:                4033   R-squared (Within):               0.6350
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9258
Time:                        12:31:42   Log-likelihood                    1309.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      435.31
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(15,3754)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             435.31
                            

In [93]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + TSD_POP_WAGE_UNDER_1250_PCT_RA \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6346
Estimator:                   PanelOLS   R-squared (Between):              0.9235
No. Observations:                4033   R-squared (Within):               0.6346
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9240
Time:                        11:13:35   Log-likelihood                    1307.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      501.74
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(13,3756)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             501.74
                            

In [94]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + TSD_POP_WAGE_UNDER_1250_PCT_TSD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6354
Estimator:                   PanelOLS   R-squared (Between):              0.8959
No. Observations:                4033   R-squared (Within):               0.6354
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.8966
Time:                        11:15:00   Log-likelihood                    1312.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      503.56
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(13,3756)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             503.56
                            

In [95]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + TSD_POP_WAGE_BETN_1250_3333_PCT_TSD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6344
Estimator:                   PanelOLS   R-squared (Between):              0.9189
No. Observations:                4033   R-squared (Within):               0.6344
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9195
Time:                        11:15:41   Log-likelihood                    1307.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      501.43
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(13,3756)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             501.43
                            

In [96]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + TSD_POP_WAGE_UNDER_3333_PCT_TSD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6354
Estimator:                   PanelOLS   R-squared (Between):              0.8865
No. Observations:                4033   R-squared (Within):               0.6354
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.8873
Time:                        11:16:05   Log-likelihood                    1312.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      503.59
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(13,3756)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             503.59
                            

In [45]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS2_HINY \
                    + YEARS_SINCE_TNC_BUS2_MIDLOW \
                    + YEARS_SINCE_TNC_RAIL2_HINY \
                    + YEARS_SINCE_TNC_RAIL2_MIDLOW \
                    + BIKE_SHARE \
                    + MDBF_Mechanical_log \
                    + scooter_flag \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6354
Estimator:                   PanelOLS   R-squared (Between):              0.9485
No. Observations:                4033   R-squared (Within):               0.6354
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9488
Time:                        10:58:56   Log-likelihood                    1312.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      408.86
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(16,3753)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             408.86
                            

In [97]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + TSD_POP_WAGE_BETN_1250_3333_PCT_RA \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6346
Estimator:                   PanelOLS   R-squared (Between):              0.9368
No. Observations:                4033   R-squared (Within):               0.6346
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9371
Time:                        11:16:31   Log-likelihood                    1307.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      501.74
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(13,3756)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             501.74
                            

In [47]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_PCT_LEHD \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS2_HINY \
                    + YEARS_SINCE_TNC_BUS2_MIDLOW \
                    + YEARS_SINCE_TNC_RAIL2_HINY \
                    + YEARS_SINCE_TNC_RAIL2_MIDLOW \
                    + BIKE_SHARE \
                    + scooter_flag \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6353
Estimator:                   PanelOLS   R-squared (Between):              0.9512
No. Observations:                4033   R-squared (Within):               0.6353
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9515
Time:                        10:59:24   Log-likelihood                    1311.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      408.62
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(16,3753)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             408.62
                            

In [98]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + TSD_POP_WAGE_UNDER_3333_PCT_RA \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6349
Estimator:                   PanelOLS   R-squared (Between):              0.9330
No. Observations:                4033   R-squared (Within):               0.6349
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9334
Time:                        11:17:04   Log-likelihood                    1309.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      502.37
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(13,3756)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             502.37
                            

In [99]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + TSD_POP_WAGE_UNDER_3333_PCT_RA \
                    + GAS_PRICE_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6295
Estimator:                   PanelOLS   R-squared (Between):              0.9913
No. Observations:                4033   R-squared (Within):               0.6295
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9914
Time:                        11:17:37   Log-likelihood                    1279.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      531.91
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(12,3757)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             531.91
                            

In [100]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_PCT_LEHD \
                    + TSD_POP_WAGE_UNDER_3333_PCT_RA \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6352
Estimator:                   PanelOLS   R-squared (Between):              0.9396
No. Observations:                4033   R-squared (Within):               0.6352
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9400
Time:                        11:18:23   Log-likelihood                    1311.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      466.94
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(14,3755)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             466.94
                            

In [101]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6347
Estimator:                   PanelOLS   R-squared (Between):              0.9380
No. Observations:                4033   R-squared (Within):               0.6347
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9384
Time:                        11:18:59   Log-likelihood                    1308.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      502.04
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(13,3756)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             502.04
                            

In [102]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6350
Estimator:                   PanelOLS   R-squared (Between):              0.9543
No. Observations:                4033   R-squared (Within):               0.6350
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9545
Time:                        11:19:46   Log-likelihood                    1310.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      466.71
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(14,3755)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             466.71
                            

In [103]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + MDBF_Total_log \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6353
Estimator:                   PanelOLS   R-squared (Between):              0.9546
No. Observations:                4033   R-squared (Within):               0.6353
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9548
Time:                        11:21:31   Log-likelihood                    1311.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      435.95
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(15,3754)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             435.95
                            

In [104]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS2_HINY \
                    + YEARS_SINCE_TNC_BUS2_MIDLOW \
                    + YEARS_SINCE_TNC_RAIL2_HINY \
                    + YEARS_SINCE_TNC_RAIL2_MIDLOW \
                    + BIKE_SHARE \
                    + MDBF_Total_log \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6349
Estimator:                   PanelOLS   R-squared (Between):              0.9527
No. Observations:                4033   R-squared (Within):               0.6349
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9530
Time:                        11:22:58   Log-likelihood                    1309.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      383.85
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(17,3752)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             383.85
                            

In [105]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS2_HINY \
                    + YEARS_SINCE_TNC_BUS2_MIDLOW \
                    + YEARS_SINCE_TNC_RAIL2_HINY \
                    + YEARS_SINCE_TNC_RAIL2_MIDLOW \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6347
Estimator:                   PanelOLS   R-squared (Between):              0.9524
No. Observations:                4033   R-squared (Within):               0.6347
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9527
Time:                        11:24:00   Log-likelihood                    1308.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      407.53
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(16,3753)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             407.53
                            

In [106]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS2_HINY \
                    + YEARS_SINCE_TNC_BUS2_MIDLOW \
                    + YEARS_SINCE_TNC_RAIL2_HINY \
                    + YEARS_SINCE_TNC_RAIL2_MIDLOW \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6344
Estimator:                   PanelOLS   R-squared (Between):              0.9381
No. Observations:                4033   R-squared (Within):               0.6344
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9385
Time:                        11:25:10   Log-likelihood                    1306.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      434.27
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(15,3754)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             434.27
                            

In [107]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS2_HINY \
                    + YEARS_SINCE_TNC_BUS2_MIDLOW \
                    + YEARS_SINCE_TNC_RAIL2_HINY \
                    + YEARS_SINCE_TNC_RAIL2_MIDLOW \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6340
Estimator:                   PanelOLS   R-squared (Between):              0.9304
No. Observations:                4033   R-squared (Within):               0.6340
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9309
Time:                        11:26:06   Log-likelihood                    1304.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      464.57
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(14,3755)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             464.57
                            

In [109]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_BUS_log \
                    + VRM_ADJ_RAIL_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS2_HINY \
                    + YEARS_SINCE_TNC_BUS2_MIDLOW \
                    + YEARS_SINCE_TNC_RAIL2_HINY \
                    + YEARS_SINCE_TNC_RAIL2_MIDLOW \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6529
Estimator:                   PanelOLS   R-squared (Between):              0.9267
No. Observations:                4033   R-squared (Within):               0.6529
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9294
Time:                        11:28:23   Log-likelihood                    1411.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      470.69
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(15,3754)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             470.69
                            

In [110]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_BUS_log \
                    + VRM_ADJ_RAIL_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS2_HINY \
                    + YEARS_SINCE_TNC_BUS2_MIDLOW \
                    + YEARS_SINCE_TNC_RAIL2_HINY \
                    + YEARS_SINCE_TNC_RAIL2_MIDLOW \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6527
Estimator:                   PanelOLS   R-squared (Between):              0.9258
No. Observations:                4041   R-squared (Within):               0.6527
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9286
Time:                        11:29:29   Log-likelihood                    1416.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      505.24
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(14,3763)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             505.24
                            

In [111]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_BUS_log \
                    + VRM_ADJ_RAIL_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6527
Estimator:                   PanelOLS   R-squared (Between):              0.9227
No. Observations:                4041   R-squared (Within):               0.6527
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9255
Time:                        11:29:59   Log-likelihood                    1415.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      589.55
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(12,3765)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             589.55
                            

In [112]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6342
Estimator:                   PanelOLS   R-squared (Between):              0.9273
No. Observations:                4041   R-squared (Within):               0.6342
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9277
Time:                        11:30:50   Log-likelihood                    1311.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      593.52
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(11,3766)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             593.52
                            

In [113]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6344
Estimator:                   PanelOLS   R-squared (Between):              0.9423
No. Observations:                4041   R-squared (Within):               0.6344
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9426
Time:                        11:31:35   Log-likelihood                    1312.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      544.54
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(12,3765)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             544.54
                            

In [114]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6341
Estimator:                   PanelOLS   R-squared (Between):              0.9401
No. Observations:                4041   R-squared (Within):               0.6341
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9404
Time:                        11:32:16   Log-likelihood                    1310.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      652.76
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(10,3767)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             652.76
                            

In [115]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6344
Estimator:                   PanelOLS   R-squared (Between):              0.9423
No. Observations:                4041   R-squared (Within):               0.6344
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9426
Time:                        11:33:04   Log-likelihood                    1312.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      544.54
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(12,3765)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             544.54
                            

In [116]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_2018',
 'HH_MEAN_INC_2018',
 'GAS_PRICE_2018',

In [117]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS_log \
                    + YEARS_SINCE_TNC_RAIL_log \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6339
Estimator:                   PanelOLS   R-squared (Between):              0.9402
No. Observations:                4041   R-squared (Within):               0.6339
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9406
Time:                        11:39:08   Log-likelihood                    1309.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      543.23
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(12,3765)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             543.23
                            

In [118]:
df['YEARS_SINCE_TNC_BUS_sqrd']=df['YEARS_SINCE_TNC_BUS']*df['YEARS_SINCE_TNC_BUS']
df['YEARS_SINCE_TNC_RAIL_sqrd']=df['YEARS_SINCE_TNC_RAIL']*df['YEARS_SINCE_TNC_RAIL']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_RAIL_sqrd \
                    + YEARS_SINCE_TNC_BUS_sqrd \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6342
Estimator:                   PanelOLS   R-squared (Between):              0.9254
No. Observations:                4041   R-squared (Within):               0.6342
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9259
Time:                        11:41:43   Log-likelihood                    1311.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      543.95
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(12,3765)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             543.95
                            

In [119]:
df['YEARS_SINCE_TNC_BUS_exp']=np.exp(df['YEARS_SINCE_TNC_BUS'])
df['YEARS_SINCE_TNC_RAIL_exp']=np.exp(df['YEARS_SINCE_TNC_RAIL'])

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS_exp \
                    + YEARS_SINCE_TNC_RAIL_exp \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6322
Estimator:                   PanelOLS   R-squared (Between):              0.8927
No. Observations:                4041   R-squared (Within):               0.6322
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.8934
Time:                        11:43:05   Log-likelihood                    1300.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      539.31
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(12,3765)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             539.31
                            

In [120]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6344
Estimator:                   PanelOLS   R-squared (Between):              0.9423
No. Observations:                4041   R-squared (Within):               0.6344
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9426
Time:                        11:44:20   Log-likelihood                    1312.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      544.54
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(12,3765)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             544.54
                            

In [123]:
df.head()

CBSA      UPT_ADJ      VRM_ADJ      FARE_ADJ  \
ID                       Year                                                  
Akron, OH Metro Area-Bus 2002  10420  5226385.660  4026372.388  5.965827e+05   
                         2003  10420  5395901.888  3976568.624  2.318587e+06   
                         2004  10420  5907162.281  4315392.555  2.279211e+06   
                         2005  10420  6013680.881  4053011.065  2.728754e+06   
                         2006  10420  6070797.419  3994786.013  2.855273e+06   

                               min_year  max_year  FARE_per_UPT  PROBLEM_FLAG  \
ID                       Year                                                   
Akron, OH Metro Area-Bus 2002      2002      2018      0.114148           NaN   
                         2003      2002      2018      0.429694           NaN   
                         2004      2002      2018      0.385839           NaN   
                         2005      2002      2018      0.453758           NaN   
                         2006      2002      2018      0.470329           NaN   

                               PROBLEM_FLAG2      Tot_Pop  Tot_Instate_Pop  \
ID                       Year                                                
Akron, OH Metro Area-Bus 2002            NaN  646583.7344      496141.0469   
                         2003            NaN  659397.0625      504631.8125   
                         2004            NaN  672492.7500      513183.7500   
                         2005            NaN  684459.0000      521491.0000   
                         2006            NaN  700943.0000      530777.0000   

                               Tot_Outstate_Pop  Tot_NonUSA_POP  \
ID                       Year                                     
Akron, OH Metro Area-Bus 2002       137193.6719      -389.53125   
                         2003       137582.3125       753.12500   
                         2004       138058.7500      1942.50000   
                         2005       138184.0000      2945.00000   
                         2006       139714.0000      4695.00000   

                               Total_Median_Income_Individual  \
ID                       Year                                   
Akron, OH Metro Area-Bus 2002                     25405.20313   
                         2003                     25120.18750   
                         2004                     24804.25000   
                         2005                     24612.00000   
                         2006                     23925.00000   

                               Native_Instate_Med_Inc_Indiv  \
ID                       Year                                 
Akron, OH Metro Area-Bus 2002                   26028.32813   
                         2003                   25632.68750   
                         2004                   25208.25000   
                         2005                   24899.00000   
                         2006                   24129.00000   

                               Native_Outstate_Med_Inc_Indiv  \
ID                       Year                                  
Akron, OH Metro Area-Bus 2002                    26651.07813   
                         2003                    25995.68750   
                         2004                    25305.25000   
                         2005                    24755.00000   
                         2006                    23644.00000   

                               Total_Pop_Poverty  Pop_Below100_Poverty  \
ID                       Year                                            
Akron, OH Metro Area-Bus 2002          682354.11                  7.55   
                         2003          682850.56                  8.81   
                         2004          683397.75                 10.08   
                         2005          683742.00                 11.30   
                         2006          684898.00                 12.70   

                    

In [122]:
#df['TNC_YEAR_FLAG_BUS']=
#df['YEARS_SINCE_TNC_RAIL_exp']=np.exp(df['YEARS_SINCE_TNC_RAIL'])

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TNC_FLAG_BUS \
                    + TNC_FLAG_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6321
Estimator:                   PanelOLS   R-squared (Between):              0.9018
No. Observations:                4041   R-squared (Within):               0.6321
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9025
Time:                        11:46:55   Log-likelihood                    1299.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      539.09
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(12,3765)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             539.09
                            

In [130]:
df=df.reset_index()


In [131]:
df['YEAR_2']=df['Year']
df=df.set_index(['ID','Year'])

In [132]:

df['TNC_YEAR_FLAG_BUS_BEFORE_2014']=np.where(df['YEAR_2']<2014, df['TNC_FLAG_BUS'],0)
df['TNC_YEAR_FLAG_BUS_2015']=np.where(df['YEAR_2']==2015, df['TNC_FLAG_BUS'],0)
df['TNC_YEAR_FLAG_BUS_2016']=np.where(df['YEAR_2']==2016, df['TNC_FLAG_BUS'],0)
df['TNC_YEAR_FLAG_BUS_2017']=np.where(df['YEAR_2']==2017, df['TNC_FLAG_BUS'],0)
df['TNC_YEAR_FLAG_BUS_2018']=np.where(df['YEAR_2']==2018, df['TNC_FLAG_BUS'],0)

df['TNC_YEAR_FLAG_RAIL_BEFORE_2014']=np.where(df['YEAR_2']<2014, df['TNC_FLAG_RAIL'],0)
df['TNC_YEAR_FLAG_RAIL_2015']=np.where(df['YEAR_2']==2015, df['TNC_FLAG_RAIL'],0)
df['TNC_YEAR_FLAG_RAIL_2016']=np.where(df['YEAR_2']==2016, df['TNC_FLAG_RAIL'],0)
df['TNC_YEAR_FLAG_RAIL_2017']=np.where(df['YEAR_2']==2017, df['TNC_FLAG_RAIL'],0)
df['TNC_YEAR_FLAG_RAIL_2018']=np.where(df['YEAR_2']==2018, df['TNC_FLAG_RAIL'],0)
#df['YEARS_SINCE_TNC_RAIL_exp']=np.exp(df['YEARS_SINCE_TNC_RAIL'])



In [133]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TNC_YEAR_FLAG_BUS_BEFORE_2014 \
                    + TNC_YEAR_FLAG_BUS_2015 \
                    + TNC_YEAR_FLAG_BUS_2016 \
                    + TNC_YEAR_FLAG_BUS_2017 \
                    + TNC_YEAR_FLAG_BUS_2018 \
                    + TNC_YEAR_FLAG_RAIL_BEFORE_2014 \
                    + TNC_YEAR_FLAG_RAIL_2015 \
                    + TNC_YEAR_FLAG_RAIL_2016 \
                    + TNC_YEAR_FLAG_RAIL_2017 \
                    + TNC_YEAR_FLAG_RAIL_2018 \
                    + TNC_FLAG_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6387
Estimator:                   PanelOLS   R-squared (Between):              0.9383
No. Observations:                4041   R-squared (Within):               0.6387
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9386
Time:                        11:56:31   Log-likelihood                    1336.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      316.11
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(21,3756)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             316.11
                            

In [139]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TNC_YEAR_FLAG_BUS_BEFORE_2014 \
                    + TNC_YEAR_FLAG_BUS_2015 \
                    + TNC_YEAR_FLAG_BUS_2016 \
                    + TNC_YEAR_FLAG_BUS_2017 \
                    + TNC_YEAR_FLAG_BUS_2018 \
                    + TNC_YEAR_FLAG_RAIL_BEFORE_2014 \
                    + TNC_YEAR_FLAG_RAIL_2015 \
                    + TNC_YEAR_FLAG_RAIL_2016 \
                    + TNC_YEAR_FLAG_RAIL_2017 \
                    + TNC_YEAR_FLAG_RAIL_2018 \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6384
Estimator:                   PanelOLS   R-squared (Between):              0.9369
No. Observations:                4041   R-squared (Within):               0.6384
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9372
Time:                        12:01:08   Log-likelihood                    1334.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      331.71
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(20,3757)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             331.71
                            

In [135]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TNC_YEAR_FLAG_BUS_BEFORE_2014 \
                    + TNC_YEAR_FLAG_BUS_2015 \
                    + TNC_YEAR_FLAG_BUS_2016 \
                    + TNC_YEAR_FLAG_BUS_2017 \
                    + TNC_YEAR_FLAG_BUS_2018 \
                    + TNC_YEAR_FLAG_RAIL_BEFORE_2014 \
                    + TNC_YEAR_FLAG_RAIL_2015 \
                    + TNC_YEAR_FLAG_RAIL_2016 \
                    + TNC_YEAR_FLAG_RAIL_2017 \
                    + TNC_YEAR_FLAG_RAIL_2018 \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + TimeEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5838
Estimator:                   PanelOLS   R-squared (Between):              0.9548
No. Observations:                4041   R-squared (Within):               0.6123
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9548
Time:                        11:58:15   Log-likelihood                    1367.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      262.35
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(20,3741)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             262.35
                            

#

In [138]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    + TimeEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5835
Estimator:                   PanelOLS   R-squared (Between):              0.9576
No. Observations:                4041   R-squared (Within):               0.6286
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9576
Time:                        11:59:44   Log-likelihood                    1366.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      437.62
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(12,3749)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             437.62
                            

In [140]:
df['YEAR_2017_FLAG']=np.where(df['YEAR_2']==2017,1,0)
df['YEAR_2018_FLAG']=np.where(df['YEAR_2']==2018,1,0)

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    + YEAR_2017_FLAG \
                    + YEAR_2018_FLAG \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6377
Estimator:                   PanelOLS   R-squared (Between):              0.9526
No. Observations:                4041   R-squared (Within):               0.6377
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9528
Time:                        12:03:45   Log-likelihood                    1330.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      473.06
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(14,3763)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             473.06
                            

In [141]:

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    + TNC_YEAR_FLAG_BUS_2017 \
                    + TNC_YEAR_FLAG_BUS_2018 \
                    + TNC_YEAR_FLAG_RAIL_2017 \
                    + TNC_YEAR_FLAG_RAIL_2018 \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6377
Estimator:                   PanelOLS   R-squared (Between):              0.9507
No. Observations:                4041   R-squared (Within):               0.6377
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9509
Time:                        12:06:36   Log-likelihood                    1330.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      413.81
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(16,3761)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             413.81
                            

In [143]:
df['TNC_YEAR_FLAG_BUS_HI_2017']=df['TNC_YEAR_FLAG_BUS_2017']*df['HI_OPEX']
df['TNC_YEAR_FLAG_BUS_NY_2017']=df['TNC_YEAR_FLAG_BUS_2017']*df['NEW_YORK']
df['TNC_YEAR_FLAG_BUS_MID_2017']=df['TNC_YEAR_FLAG_BUS_2017']*df['MID_OPEX']
df['TNC_YEAR_FLAG_BUS_LOW_2017']=df['TNC_YEAR_FLAG_BUS_2017']*df['LOW_OPEX']

df['TNC_YEAR_FLAG_BUS_HINY_2017']=df['TNC_YEAR_FLAG_BUS_HI_2017']+df['TNC_YEAR_FLAG_BUS_NY_2017']
df['TNC_YEAR_FLAG_BUS_MIDLOW_2017']=df['TNC_YEAR_FLAG_BUS_MID_2017']+df['TNC_YEAR_FLAG_BUS_LOW_2017']

df['TNC_YEAR_FLAG_BUS_HI_2018']=df['TNC_YEAR_FLAG_BUS_2018']*df['HI_OPEX']
df['TNC_YEAR_FLAG_BUS_NY_2018']=df['TNC_YEAR_FLAG_BUS_2018']*df['NEW_YORK']
df['TNC_YEAR_FLAG_BUS_MID_2018']=df['TNC_YEAR_FLAG_BUS_2018']*df['MID_OPEX']
df['TNC_YEAR_FLAG_BUS_LOW_2018']=df['TNC_YEAR_FLAG_BUS_2018']*df['LOW_OPEX']

df['TNC_YEAR_FLAG_BUS_HINY_2018']=df['TNC_YEAR_FLAG_BUS_HI_2018']+df['TNC_YEAR_FLAG_BUS_NY_2018']
df['TNC_YEAR_FLAG_BUS_MIDLOW_2018']=df['TNC_YEAR_FLAG_BUS_MID_2018']+df['TNC_YEAR_FLAG_BUS_LOW_2018']

df['TNC_YEAR_FLAG_RAIL_HI_2017']=df['TNC_YEAR_FLAG_RAIL_2017']*df['HI_OPEX']
df['TNC_YEAR_FLAG_RAIL_NY_2017']=df['TNC_YEAR_FLAG_RAIL_2017']*df['NEW_YORK']
df['TNC_YEAR_FLAG_RAIL_MID_2017']=df['TNC_YEAR_FLAG_RAIL_2017']*df['MID_OPEX']
df['TNC_YEAR_FLAG_RAIL_LOW_2017']=df['TNC_YEAR_FLAG_RAIL_2017']*df['LOW_OPEX']

df['TNC_YEAR_FLAG_RAIL_HINY_2017']=df['TNC_YEAR_FLAG_RAIL_HI_2017']+df['TNC_YEAR_FLAG_RAIL_NY_2017']
df['TNC_YEAR_FLAG_RAIL_MIDLOW_2017']=df['TNC_YEAR_FLAG_RAIL_MID_2017']+df['TNC_YEAR_FLAG_RAIL_LOW_2017']

df['TNC_YEAR_FLAG_RAIL_HI_2018']=df['TNC_YEAR_FLAG_RAIL_2018']*df['HI_OPEX']
df['TNC_YEAR_FLAG_RAIL_NY_2018']=df['TNC_YEAR_FLAG_RAIL_2018']*df['NEW_YORK']
df['TNC_YEAR_FLAG_RAIL_MID_2018']=df['TNC_YEAR_FLAG_RAIL_2018']*df['MID_OPEX']
df['TNC_YEAR_FLAG_RAIL_LOW_2018']=df['TNC_YEAR_FLAG_RAIL_2018']*df['LOW_OPEX']

df['TNC_YEAR_FLAG_RAIL_HINY_2018']=df['TNC_YEAR_FLAG_RAIL_HI_2018']+df['TNC_YEAR_FLAG_RAIL_NY_2018']
df['TNC_YEAR_FLAG_RAIL_MIDLOW_2018']=df['TNC_YEAR_FLAG_RAIL_MID_2018']+df['TNC_YEAR_FLAG_RAIL_LOW_2018']

In [144]:


mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    + TNC_YEAR_FLAG_BUS_HINY_2017 \
                    + TNC_YEAR_FLAG_BUS_HINY_2018 \
                    + TNC_YEAR_FLAG_BUS_MIDLOW_2017 \
                    + TNC_YEAR_FLAG_BUS_MIDLOW_2018 \
                    + TNC_YEAR_FLAG_RAIL_HINY_2017 \
                    + TNC_YEAR_FLAG_RAIL_HINY_2018 \
                    + TNC_YEAR_FLAG_RAIL_MIDLOW_2017 \
                    + TNC_YEAR_FLAG_RAIL_MIDLOW_2018 \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6378
Estimator:                   PanelOLS   R-squared (Between):              0.9501
No. Observations:                4041   R-squared (Within):               0.6378
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9504
Time:                        12:17:54   Log-likelihood                    1331.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      330.82
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(20,3757)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             330.82
                            

In [145]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_2018',
 'HH_MEAN_INC_2018',
 'GAS_PRICE_2018',

In [146]:
df['YEARS_AFTER_2014'] = df['YEAR_2'].apply(lambda x : max(x-2014, 0))
df['TNC_YEARS_AFTER_2014'] = df['YEARS_AFTER_2014'] * df['TNC_FLAG']

df['TNC_YEARS_AFTER_2014_BUS'] = df['TNC_YEARS_AFTER_2014'] * df['BUS_FLAG']
df['TNC_YEARS_AFTER_2014_RAIL'] = df['TNC_YEARS_AFTER_2014'] * df['RAIL_FLAG']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    + TNC_YEARS_AFTER_2014_BUS \
                    + TNC_YEARS_AFTER_2014_RAIL \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6362
Estimator:                   PanelOLS   R-squared (Between):              0.9526
No. Observations:                4041   R-squared (Within):               0.6362
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9529
Time:                        12:26:27   Log-likelihood                    1322.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      470.10
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(14,3763)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             470.10
                            

In [147]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    + TNC_YEARS_AFTER_2014_BUS \
                    + TNC_YEARS_AFTER_2014_RAIL \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6351
Estimator:                   PanelOLS   R-squared (Between):              0.9548
No. Observations:                4041   R-squared (Within):               0.6351
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9550
Time:                        12:27:36   Log-likelihood                    1316.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      546.12
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(12,3765)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             546.12
                            

In [ ]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    + TNC_YEARS_AFTER_2014_BUS \
                    + TNC_YEARS_AFTER_2014_RAIL \
                    ',data=df)
res=mod.fit()
print(res)

In [148]:
df['TNC_YEARS_AFTER_2014_BUS'] = df['TNC_YEARS_AFTER_2014'] * df['BUS_FLAG']
df['TNC_YEARS_AFTER_2014_RAIL'] = df['TNC_YEARS_AFTER_2014'] * df['RAIL_FLAG']


mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    + TNC_YEARS_AFTER_2014_BUS \
                    + TNC_YEARS_AFTER_2014_RAIL \
                    ',data=df)
res=mod.fit()
print(res)

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [150]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    + YEARS_SINCE_TNC_BUS * YEARS_AFTER_2014\
                    + YEARS_SINCE_TNC_RAIL * YEARS_AFTER_2014\
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6365
Estimator:                   PanelOLS   R-squared (Between):              0.9500
No. Observations:                4041   R-squared (Within):               0.6365
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9503
Time:                        12:34:19   Log-likelihood                    1324.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      439.22
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(15,3762)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             439.22
                            

In [151]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    + YEARS_AFTER_2014 \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6355
Estimator:                   PanelOLS   R-squared (Between):              0.9575
No. Observations:                4041   R-squared (Within):               0.6355
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9578
Time:                        12:36:26   Log-likelihood                    1318.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      504.88
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(13,3764)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             504.88
                            

In [152]:
#Preferred model 

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6344
Estimator:                   PanelOLS   R-squared (Between):              0.9423
No. Observations:                4041   R-squared (Within):               0.6344
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9426
Time:                        12:37:47   Log-likelihood                    1312.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      544.54
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(12,3765)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             544.54
                            

In [239]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_2018',
 'HH_MEAN_INC_2018',
 'GAS_PRICE_2018',

In [241]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_POP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    + UBER_REV_SCALED_BUS_HINY \
                    + UBER_REV_SCALED_BUS_MIDLOW \
                    + UBER_REV_SCALED_RAIL_HINY \
                    + UBER_REV_SCALED_RAIL_MIDLOW \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6345
Estimator:                   PanelOLS   R-squared (Between):              0.9156
No. Observations:                4041   R-squared (Within):               0.6345
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9161
Time:                        01:51:35   Log-likelihood                    1312.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      466.61
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.307   Distribution:                 F(14,3763)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             466.61
                            

In [108]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_EMP_PCT_LEHD \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS2_HINY \
                    + YEARS_SINCE_TNC_BUS2_MIDLOW \
                    + YEARS_SINCE_TNC_RAIL2_HINY \
                    + YEARS_SINCE_TNC_RAIL2_MIDLOW \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6341
Estimator:                   PanelOLS   R-squared (Between):              0.9394
No. Observations:                4033   R-squared (Within):               0.6341
Date:                Fri, Apr 17 2020   R-squared (Overall):              0.9398
Time:                        11:26:19   Log-likelihood                    1305.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      464.87
Entities:                         264   P-value                           0.0000
Avg Obs:                       15.277   Distribution:                 F(14,3755)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             464.87
                            

In [54]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_LABOR_FORCE_PCT \
                    + TSD_EMP_PCT \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6312
Estimator:                   PanelOLS   R-squared (Between):              0.7393
No. Observations:                4018   R-squared (Within):               0.6312
Date:                Fri, Apr 10 2020   R-squared (Overall):              0.7417
Time:                        03:55:57   Log-likelihood                    1254.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      534.09
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.336   Distribution:                 F(12,3744)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             534.09
                            

In [55]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + TSD_LABOR_FORCE_PCT \
                    + TSD_EMP_PCT \
                    + GAS_PRICE_2018_log \
                    + JOBS_LESS_THAN_40K_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6249
Estimator:                   PanelOLS   R-squared (Between):              0.9613
No. Observations:                4016   R-squared (Within):               0.6249
Date:                Fri, Apr 10 2020   R-squared (Overall):              0.9613
Time:                        03:58:52   Log-likelihood                    1218.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      519.43
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.328   Distribution:                 F(12,3742)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             519.43
                            

In [38]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + TSD_LABOR_FORCE_PCT \
                    + TSD_EMP_PCT \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6335
Estimator:                   PanelOLS   R-squared (Between):              0.9284
No. Observations:                4018   R-squared (Within):               0.6335
Date:                Fri, Apr 10 2020   R-squared (Overall):              0.9289
Time:                        11:29:06   Log-likelihood                    1266.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      497.65
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.336   Distribution:                 F(13,3743)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             497.65
                            

/Users/rianajawad/anaconda3/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [39]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + TSD_EMP_PCT \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6333
Estimator:                   PanelOLS   R-squared (Between):              0.9280
No. Observations:                4018   R-squared (Within):               0.6333
Date:                Fri, Apr 10 2020   R-squared (Overall):              0.9284
Time:                        11:29:57   Log-likelihood                    1266.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      538.95
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.336   Distribution:                 F(12,3744)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             538.95
                            

In [40]:
#Preferred model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + TSD_LABOR_FORCE_PCT \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6329
Estimator:                   PanelOLS   R-squared (Between):              0.9192
No. Observations:                4018   R-squared (Within):               0.6329
Date:                Fri, Apr 10 2020   R-squared (Overall):              0.9198
Time:                        11:30:36   Log-likelihood                    1263.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      538.01
Entities:                         262   P-value                           0.0000
Avg Obs:                       15.336   Distribution:                 F(12,3744)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             538.01
                            

In [41]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6249
Estimator:                   PanelOLS   R-squared (Between):              0.9274
No. Observations:                4155   R-squared (Within):               0.6249
Date:                Fri, Apr 10 2020   R-squared (Overall):              0.9277
Time:                        11:36:33   Log-likelihood                    1278.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      586.20
Entities:                         273   P-value                           0.0000
Avg Obs:                       15.220   Distribution:                 F(11,3871)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             586.20
                            

In [44]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre_x',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_2018',
 'HH_MEAN_INC_2018',
 'GAS_PRICE_2018

In [36]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS2_HINY \
                    + YEARS_SINCE_TNC_BUS2_MIDLOW \
                    + YEARS_SINCE_TNC_RAIL2_HINY \
                    + YEARS_SINCE_TNC_RAIL2_MIDLOW \
                    + BIKE_SHARE \
                    + MDBF_Total_log \
                    + scooter_flag \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6283
Estimator:                   PanelOLS   R-squared (Between):              0.9426
No. Observations:                4145   R-squared (Within):               0.6283
Date:                Tue, Mar 31 2020   R-squared (Overall):              0.9433
Time:                        13:34:24   Log-likelihood                    1309.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      434.67
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(15,3858)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             434.67
                            

/Users/rianajawad/anaconda3/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [169]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS2_HINY \
                    + YEARS_SINCE_TNC_BUS2_MIDLOW \
                    + YEARS_SINCE_TNC_RAIL2_HINY \
                    + YEARS_SINCE_TNC_RAIL2_MIDLOW \
                    + BIKE_SHARE \
                    + MDBF_Mechanical_log \
                    + scooter_flag \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6249
Estimator:                   PanelOLS   R-squared (Between):              0.9325
No. Observations:                4155   R-squared (Within):               0.6249
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9329
Time:                        09:42:40   Log-likelihood                    1278.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      460.36
Entities:                         273   P-value                           0.0000
Avg Obs:                       15.220   Distribution:                 F(14,3868)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             460.36
                            

In [170]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS2_HINY \
                    + YEARS_SINCE_TNC_BUS2_MIDLOW \
                    + YEARS_SINCE_TNC_RAIL2_HINY \
                    + YEARS_SINCE_TNC_RAIL2_MIDLOW \
                    + BIKE_SHARE \
                    + MDBF_Total_log \
                    + scooter_flag \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6301
Estimator:                   PanelOLS   R-squared (Between):              0.9705
No. Observations:                4145   R-squared (Within):               0.6301
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9710
Time:                        09:42:55   Log-likelihood                    1319.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      438.16
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(15,3858)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             438.16
                            

In [171]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS2_HINY \
                    + YEARS_SINCE_TNC_BUS2_MIDLOW \
                    + YEARS_SINCE_TNC_RAIL2_HINY \
                    + YEARS_SINCE_TNC_RAIL2_MIDLOW \
                    + BIKE_SHARE \
                    + scooter_flag \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6300
Estimator:                   PanelOLS   R-squared (Between):              0.9708
No. Observations:                4145   R-squared (Within):               0.6300
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9712
Time:                        09:43:08   Log-likelihood                    1318.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      469.26
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(14,3859)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             469.26
                            

In [172]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6305
Estimator:                   PanelOLS   R-squared (Between):              0.9641
No. Observations:                4145   R-squared (Within):               0.6305
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9647
Time:                        09:43:18   Log-likelihood                    1321.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      549.04
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(12,3861)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             549.04
                            

In [37]:
df['YEARS_SINCE_TNC_BUS_SQRD']=df['YEARS_SINCE_TNC_BUS']*df['YEARS_SINCE_TNC_BUS']
df['YEARS_SINCE_TNC_RAIL_SQRD']=df['YEARS_SINCE_TNC_RAIL']*df['YEARS_SINCE_TNC_RAIL']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS_SQRD \
                    + YEARS_SINCE_TNC_RAIL_SQRD \
                    + BIKE_SHARE \
                    + scooter_flag \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6301
Estimator:                   PanelOLS   R-squared (Between):              0.9748
No. Observations:                4145   R-squared (Within):               0.6301
Date:                Tue, Mar 31 2020   R-squared (Overall):              0.9753
Time:                        13:35:04   Log-likelihood                    1319.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      548.08
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(12,3861)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             548.08
                            

In [174]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + YEARS_SINCE_TNC_BUS_SQRD \
                    + YEARS_SINCE_TNC_RAIL_SQRD \
                    + BIKE_SHARE \
                    + scooter_flag \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6309
Estimator:                   PanelOLS   R-squared (Between):              0.9646
No. Observations:                4145   R-squared (Within):               0.6309
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9652
Time:                        09:43:36   Log-likelihood                    1323.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      471.08
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(14,3859)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             471.08
                            

In [175]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + YEARS_SINCE_TNC_BUS_SQRD \
                    + YEARS_SINCE_TNC_RAIL_SQRD \
                    + BIKE_SHARE \
                    + scooter_flag \
                    + EntityEffects \
                    + MDBF_Total_log \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6310
Estimator:                   PanelOLS   R-squared (Between):              0.9645
No. Observations:                4145   R-squared (Within):               0.6310
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9651
Time:                        09:43:46   Log-likelihood                    1324.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      439.87
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(15,3858)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             439.87
                            

In [176]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + YEARS_SINCE_TNC_BUS_SQRD \
                    + YEARS_SINCE_TNC_RAIL_SQRD \
                    + BIKE_SHARE \
                    + scooter_flag \
                    + EntityEffects \
                    + MDBF_Total \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6309
Estimator:                   PanelOLS   R-squared (Between):              0.9652
No. Observations:                4145   R-squared (Within):               0.6309
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9658
Time:                        09:44:13   Log-likelihood                    1324.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      439.66
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(15,3858)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             439.66
                            

In [177]:
def alt_mode(df):
    if (df['BIKE_SHARE']==1) or (df['scooter_flag']==1) or (df['TNC_FLAG']==1):
        x=1
    else:
        x=0
    return x
    
df['ALT_MODE']=df.apply(alt_mode,axis=1)
df['FAILURE_ALT_MODE_INTERACTION']=df['ALT_MODE']*df['MDBF_Total']
df['FAILURE_ALT_MODE_INTERACTION_log'] = np.log(df['FAILURE_ALT_MODE_INTERACTION']+1)

In [178]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + YEARS_SINCE_TNC_BUS_SQRD \
                    + YEARS_SINCE_TNC_RAIL_SQRD \
                    + BIKE_SHARE \
                    + scooter_flag \
                    + EntityEffects \
                    + FAILURE_ALT_MODE_INTERACTION_log  \
                    ',data=df)
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6309
Estimator:                   PanelOLS   R-squared (Between):              0.9667
No. Observations:                4145   R-squared (Within):               0.6309
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9673
Time:                        09:45:43   Log-likelihood                    1323.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      439.64
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(15,3858)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             439.64
                            

In [179]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + YEARS_SINCE_TNC_BUS_SQRD \
                    + YEARS_SINCE_TNC_RAIL_SQRD \
                    + BIKE_SHARE \
                    + scooter_flag \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6309
Estimator:                   PanelOLS   R-squared (Between):              0.9646
No. Observations:                4145   R-squared (Within):               0.6309
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9652
Time:                        09:47:13   Log-likelihood                    1323.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      471.08
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(14,3859)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             471.08
                            

In [180]:
## Create FACs for this model as well

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + YEARS_SINCE_TNC_BUS_SQRD \
                    + BIKE_SHARE \
                    + scooter_flag \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6306
Estimator:                   PanelOLS   R-squared (Between):              0.9636
No. Observations:                4145   R-squared (Within):               0.6306
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9642
Time:                        09:47:32   Log-likelihood                    1322.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      506.90
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(13,3860)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             506.90
                            

In [182]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag \
                    + EntityEffects \
                    + FAILURE_ALT_MODE_INTERACTION_log  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6306
Estimator:                   PanelOLS   R-squared (Between):              0.9660
No. Observations:                4145   R-squared (Within):               0.6306
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9666
Time:                        09:47:54   Log-likelihood                    1322.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      506.78
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(13,3860)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             506.78
                            

In [183]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + FAILURE_ALT_MODE_INTERACTION_log  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6300
Estimator:                   PanelOLS   R-squared (Between):              0.9744
No. Observations:                4145   R-squared (Within):               0.6300
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9749
Time:                        09:48:06   Log-likelihood                    1319.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      547.94
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(12,3861)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             547.94
                            

In [184]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + MDBF_Total_log  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6301
Estimator:                   PanelOLS   R-squared (Between):              0.9723
No. Observations:                4145   R-squared (Within):               0.6301
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9727
Time:                        09:48:11   Log-likelihood                    1319.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      548.06
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(12,3861)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             548.06
                            

In [38]:
### preferred model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6305
Estimator:                   PanelOLS   R-squared (Between):              0.9641
No. Observations:                4145   R-squared (Within):               0.6305
Date:                Tue, Mar 31 2020   R-squared (Overall):              0.9647
Time:                        13:35:39   Log-likelihood                    1321.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      549.04
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(12,3861)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             549.04
                            

In [39]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_2018',
 'HH_MEAN_INC_2018',
 'GAS_PRICE_2018',

In [41]:
### preferred model updated with new emp
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + TOT_EMP_MSA_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6302
Estimator:                   PanelOLS   R-squared (Between):              0.9736
No. Observations:                4145   R-squared (Within):               0.6302
Date:                Tue, Mar 31 2020   R-squared (Overall):              0.9740
Time:                        13:40:15   Log-likelihood                    1319.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      548.20
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(12,3861)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             548.20
                            

In [208]:
## Create FACs for this model as well

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + YEARS_SINCE_TNC_BUS_SQRD \
                    + BIKE_SHARE \
                    + scooter_flag \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6306
Estimator:                   PanelOLS   R-squared (Between):              0.9636
No. Observations:                4145   R-squared (Within):               0.6306
Date:                Tue, Mar 24 2020   R-squared (Overall):              0.9642
Time:                        12:53:59   Log-likelihood                    1322.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      506.90
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(13,3860)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             506.90
                            

In [42]:
## Create FACs for this model as well

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + TOT_EMP_MSA_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + YEARS_SINCE_TNC_BUS_SQRD \
                    + BIKE_SHARE \
                    + scooter_flag \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6303
Estimator:                   PanelOLS   R-squared (Between):              0.9724
No. Observations:                4145   R-squared (Within):               0.6303
Date:                Mon, Apr 06 2020   R-squared (Overall):              0.9729
Time:                        02:18:01   Log-likelihood                    1320.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      506.25
Entities:                         272   P-value                           0.0000
Avg Obs:                       15.239   Distribution:                 F(13,3860)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             506.25
                            

In [186]:
# mod=PanelOLS.from_formula('UPT_ADJ_log \
#                     ~ VRM_ADJ_log \
#                     + FARE_per_UPT_2018_log \
#                     + Tot_Pop_log \
#                     + GAS_PRICE_2018_log \
#                     + TOTAL_MED_INC_INDIV_2018_log \
#                     + PCT_HH_NO_VEH \
#                     + WEIGHTED_POP_DENSITY_log \
#                     + JTW_HOME_PCT \
#                     + YEARS_SINCE_TNC_BUS2_HINY \
#                     + YEARS_SINCE_TNC_BUS2_MIDLOW \
#                     + YEARS_SINCE_TNC_RAIL2_HINY \
#                     + YEARS_SINCE_TNC_RAIL2_MIDLOW \
#                     + BIKE_SHARE \
#                     + scooter_flag \
#                     + EntityEffects \
#                     ',data=df)
# res=mod.fit()
# print(res)

In [187]:
# mod=PanelOLS.from_formula('UPT_ADJ_log \
#                     ~ VRM_ADJ_log \
#                     + FARE_per_UPT_2018_log \
#                     + TOT_EMP_MSA_log \
#                     + GAS_PRICE_2018_log \
#                     + TOTAL_MED_INC_INDIV_2018_log \
#                     + PCT_HH_NO_VEH \
#                     + WEIGHTED_POP_DENSITY_log \
#                     + JTW_HOME_PCT \
#                     + YEARS_SINCE_TNC_BUS2_HINY \
#                     + YEARS_SINCE_TNC_BUS2_MIDLOW \
#                     + YEARS_SINCE_TNC_RAIL2_HINY \
#                     + YEARS_SINCE_TNC_RAIL2_MIDLOW \
#                     + BIKE_SHARE \
#                     + scooter_flag \
#                     + EntityEffects \
#                     ',data=df)
# res=mod.fit()
# print(res)

In [188]:
# mod=PanelOLS.from_formula('UPT_ADJ_log \
#                     ~ VRM_ADJ_log \
#                     + FARE_per_UPT_2018_log \
#                     + Tot_Pop_log \
#                     + TOT_EMP_MSA_log \
#                     + GAS_PRICE_2018_log \
#                     + TOTAL_MED_INC_INDIV_2018_log \
#                     + PCT_HH_NO_VEH \
#                     + WEIGHTED_POP_DENSITY_log \
#                     + JTW_HOME_PCT \
#                     + YEARS_SINCE_TNC_BUS2_HINY \
#                     + YEARS_SINCE_TNC_BUS2_MIDLOW \
#                     + YEARS_SINCE_TNC_RAIL2_HINY \
#                     + YEARS_SINCE_TNC_RAIL2_MIDLOW \
#                     + BIKE_SHARE \
#                     + scooter_flag \
#                     + EntityEffects \
#                     ',data=df)
# res=mod.fit()
# print(res)

In [189]:
# mod=PanelOLS.from_formula('UPT_ADJ_log \
#                     ~ VRM_ADJ_log \
#                     + FARE_per_UPT_2018_log \
#                     + Tot_Pop_log \
#                     + UNEMP_RATE_PCT \
#                     + GAS_PRICE_2018_log \
#                     + TOTAL_MED_INC_INDIV_2018_log \
#                     + PCT_HH_NO_VEH \
#                     + WEIGHTED_POP_DENSITY_log \
#                     + JTW_HOME_PCT \
#                     + YEARS_SINCE_TNC_BUS2_HINY \
#                     + YEARS_SINCE_TNC_BUS2_MIDLOW \
#                     + YEARS_SINCE_TNC_RAIL2_HINY \
#                     + YEARS_SINCE_TNC_RAIL2_MIDLOW \
#                     + BIKE_SHARE \
#                     + scooter_flag \
#                     + EntityEffects \
#                     ',data=df)
# res=mod.fit()
# print(res)

In [190]:
# mod=PanelOLS.from_formula('UPT_ADJ_log \
#                     ~ VRM_ADJ_log \
#                     + FARE_per_UPT_2018_log \
#                     + POP_EMP_log \
#                     + WEIGHTED_POP_DENSITY \
#                     + GAS_PRICE_2018_log \
#                     + TOTAL_MED_INC_INDIV_2018_log \
#                     + PCT_HH_NO_VEH \
#                     + JTW_HOME_PCT \
#                     + YEARS_SINCE_TNC_BUS2_HINY \
#                     + YEARS_SINCE_TNC_BUS2_MIDLOW \
#                     + YEARS_SINCE_TNC_RAIL2_HINY \
#                     + YEARS_SINCE_TNC_RAIL2_MIDLOW \
#                     + BIKE_SHARE \
#                     + scooter_flag \
#                     + EntityEffects \
#                     ',data=df)
# res=mod.fit()
# print(res)

In [191]:
# mod=PanelOLS.from_formula('UPT_ADJ_log \
#                     ~ VRM_ADJ_log \
#                     + FARE_per_UPT_2018_log \
#                     + POP_EMP_log \
#                     + TSD_POP_PCT \
#                     + GAS_PRICE_2018_log \
#                     + TOTAL_MED_INC_INDIV_2018_log \
#                     + PCT_HH_NO_VEH \
#                     + JTW_HOME_PCT \
#                     + YEARS_SINCE_TNC_BUS2_HINY \
#                     + YEARS_SINCE_TNC_BUS2_MIDLOW \
#                     + YEARS_SINCE_TNC_RAIL2_HINY \
#                     + YEARS_SINCE_TNC_RAIL2_MIDLOW \
#                     + BIKE_SHARE \
#                     + scooter_flag \
#                     + EntityEffects \
#                     ',data=df)
# res=mod.fit()
# print(res)

In [192]:
# def alt_mode(df):
#     if (df['BIKE_SHARE']==1) or (df['scooter_flag']==1) or (df['TNC_FLAG']==1):
#         x=1
#     else:
#         x=0
#     return x
    
# df['ALT_MODE']=df.apply(alt_mode,axis=1)
# df['FAILURE_ALT_MODE_INTERACTION']=df['ALT_MODE']*df['MDBF_Total']
# df['FAILURE_ALT_MODE_INTERACTION_log'] = np.log(df['FAILURE_ALT_MODE_INTERACTION']+1)

In [193]:
# df['FAILURE_ALT_MODE_INTERACTION']=df['ALT_MODE']*df['MDBF_Total']
# df['FAILURE_ALT_MODE_INTERACTION_log'] = np.log(df['FAILURE_ALT_MODE_INTERACTION']+1)

In [194]:
# mod=PanelOLS.from_formula('UPT_ADJ_log \
#                     ~ VRM_ADJ_log \
#                     + FARE_per_UPT_2018_log \
#                     + POP_EMP_log \
#                     + WEIGHTED_POP_DENSITY \
#                     + GAS_PRICE_2018_log \
#                     + TOTAL_MED_INC_INDIV_2018_log \
#                     + PCT_HH_NO_VEH \
#                     + JTW_HOME_PCT \
#                     + YEARS_SINCE_TNC_BUS2_HINY \
#                     + YEARS_SINCE_TNC_BUS2_MIDLOW \
#                     + YEARS_SINCE_TNC_RAIL2_HINY \
#                     + YEARS_SINCE_TNC_RAIL2_MIDLOW \
#                     + BIKE_SHARE \
#                     + scooter_flag \
#                     + FAILURE_ALT_MODE_INTERACTION_log \
#                     + EntityEffects \
#                     ',data=df)
# res=mod.fit()
# print(res)

# Calculate FAC

In [209]:
# keep the relevant fields in the data set
app_data = pd.concat([df[['RAIL_FLAG', 'CLUSTER_APTA']], 
                      res.model.dependent.dataframe, 
                      res.model.exog.dataframe, 
                      res.estimated_effects, 
                      res.resids], axis=1)

In [210]:
# calculate the fitted values
app_data['fitted'] = app_data['estimated_effects']
for var in res.params.keys(): 
    app_data['fitted'] = app_data['fitted'] + (res.params[var] * app_data[var])

# and check that we add up correctly
app_data['check_fitted'] = app_data['fitted'] + app_data['residual'] - app_data['UPT_ADJ_log']

In [211]:
# create the necessary columns

# dependents
app_data['UPT_ADJ']    = np.exp(app_data['UPT_ADJ_log']) - 1
app_data['fitted_exp'] = np.exp(app_data['fitted']) - 1

# linear versions of logged variables
for var in res.params.keys(): 
    if '_log' in var: 
        app_data[var.replace('_log', '')] = np.exp(app_data[var]) - 1              

# ratios and differences
for var in res.params.keys(): 
    if '_log' in var: 
        app_data[var.replace('_log', '_ratio')] = 1
    else: 
        app_data[var+'_diff'] = 0

# FAC multipliers
for var in res.params.keys(): 
    app_data[var+'_FAC_ratio'] = 1

app_data['UPT_ADJ_base'] = 0
app_data['UPT_ADJ_ratio'] = 1
app_data['fitted_exp_ratio'] = 1
app_data['UPT_ADJ_diff'] = 0
app_data['fitted_exp_diff'] = 0

# for tracking new systems
app_data['UPT_ADJ_first_year'] = 0
app_data['UPT_ADJ_new_reporter'] = 0



In [212]:
# get the ridership in the first year the system starts. This allows us to track new systems. 
# calculate the FAC relative to a specific base year
ids = app_data.index.get_level_values(0).unique()

for id in ids: 
    years = app_data.loc[id].index.get_level_values(0).sort_values()
    first_year = years[0]

    app_data.loc[(id,first_year),'UPT_ADJ_new_reporter'] = app_data.loc[(id,first_year),'UPT_ADJ']
    
    for year in years: 
        app_data.loc[(id,year),'UPT_ADJ_first_year']   = app_data.loc[(id,first_year),'UPT_ADJ']

In [213]:
# calculate the FAC relative to a specific base year
ids = app_data.index.get_level_values(0).unique()

for id in ids: 
    years = app_data.loc[id].index.get_level_values(0).sort_values()

    for year in years[1:]:       
        base_year = year-1
        
        for var in res.params.keys(): 
            # ratios and differences
            if '_log' in var: 
                out_var = var.replace('_log', '_ratio')
                app_data.loc[(id,year), out_var] = (app_data.loc[(id,year), var] 
                                                 / app_data.loc[(id,base_year), var])
            else: 
                out_var = var+'_diff'                    
                app_data.loc[(id,year), out_var] = (app_data.loc[(id,year), var] 
                                                 - app_data.loc[(id,base_year), var])

            # FAC multipliers
            app_data.loc[(id,year),var+'_FAC_ratio'] = np.exp(res.params[var] * (
                                                    app_data.loc[(id,year), var] 
                                                    - app_data.loc[(id,base_year), var]))

            # estimated effects (if time effects is zero, no change)
            app_data.loc[(id,year),'effects_FAC_ratio'] = np.exp(
                                                    app_data.loc[(id,year), 'estimated_effects'] 
                                                    - app_data.loc[(id,base_year), 'estimated_effects'])

            # residual
            app_data.loc[(id,year),'residual_FAC_ratio'] = np.exp(
                                                    app_data.loc[(id,year), 'residual'] 
                                                    - app_data.loc[(id,base_year), 'residual'])

        # observed and fitted changes            
        app_data.loc[(id,year),'UPT_ADJ_base'] = app_data.loc[(id,base_year),'UPT_ADJ']
        app_data.loc[(id,year),'UPT_ADJ_ratio'] = (app_data.loc[(id,year),'UPT_ADJ'] 
                                                 / app_data.loc[(id,base_year),'UPT_ADJ'])
        app_data.loc[(id,year),'fitted_exp_ratio'] = (app_data.loc[(id,year),'fitted_exp'] 
                                                 / app_data.loc[(id,base_year),'fitted_exp'])

        app_data.loc[(id,year),'UPT_ADJ_diff'] = (app_data.loc[(id,year),'UPT_ADJ'] 
                                                 - app_data.loc[(id,base_year),'UPT_ADJ'])
        app_data.loc[(id,year),'fitted_exp_diff'] = (app_data.loc[(id,year),'fitted_exp'] 
                                                 - app_data.loc[(id,base_year),'fitted_exp'])
        

/Users/rianajawad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
/Users/rianajawad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app


In [214]:
# calculate the FAC
app_data['FAC_Sum'] = 0
for var in res.params.keys(): 
    app_data[var+'_FAC'] = (app_data[var+'_FAC_ratio'] - 1) * app_data['UPT_ADJ_base']
    app_data['FAC_Sum'] = app_data['FAC_Sum'] + app_data[var+'_FAC']



In [215]:

# total FAC is based on the fitted model (applied multiplicitively)
app_data['Known_FAC'] = (app_data['fitted_exp_ratio'] - 1) * app_data['UPT_ADJ_base']

# uknown change is the difference between the observed change and the known change
app_data['Unknown_FAC'] = app_data['UPT_ADJ_diff'] - app_data['Known_FAC']

# the change in ridership associated new systems
app_data['New_Reporter_FAC'] = app_data['UPT_ADJ_new_reporter'] 

# should be teh same as UPT_ADJ_DIFF
app_data['Total_Change'] = app_data['Known_FAC'] + app_data['Unknown_FAC'] + app_data['New_Reporter_FAC']

In [216]:
# calculate APTA - 4 groups
app_data['CLUSTER_APTA4'] = np.floor(app_data['CLUSTER_APTA']/10)

In [217]:
# reset the index so I get ID and year
app_data = app_data.reset_index()
app_data = app_data.rename(columns={'level_0' : 'ID', 'level_1' : 'Year'})
app_data.to_csv('FAC_Model2.csv')

In [218]:
# these are the fields we keep
out_fields = ['ID', 'Year', 'RAIL_FLAG', 
              'CLUSTER_APTA', 'CLUSTER_APTA4',    
              'UPT_ADJ_first_year', 
              'UPT_ADJ_base', 'UPT_ADJ', 'UPT_ADJ_diff', 
              'fitted_exp', 'fitted_exp_diff']

# exogenous variables 
for var in res.params.keys(): 
    out_var = var.replace('_log', '')
    out_fields = out_fields + [out_var]
    
# FAC
for var in res.params.keys(): 
    out_fields = out_fields + [var+'_FAC']
out_fields = out_fields + ['FAC_Sum', 'Known_FAC', 'Unknown_FAC', 'New_Reporter_FAC', 'Total_Change']
    
# keep those fields
summary_data = app_data.reset_index()[out_fields]

In [219]:
# take weighted average of exogenous variables
for var in res.params.keys(): 
    out_var = var.replace('_log', '')
    summary_data[out_var] = summary_data[out_var] * summary_data['UPT_ADJ_first_year']

# aggregate to bus/rail totals
summary_data_apta4 = summary_data.groupby(by=['CLUSTER_APTA4', 'RAIL_FLAG', 'Year']).agg('sum')

# divide for weighted averages
for var in res.params.keys(): 
    out_var = var.replace('_log', '')
    summary_data_apta4[out_var] = summary_data_apta4[out_var] / summary_data_apta4['UPT_ADJ_first_year']

In [220]:

summary_data_apta4.to_csv('FAC_totals_APTA4_CLUSTERS_Model2.csv')

